# Part-based R-CNNs for Fine-grained Category Detection  


**首先利用 Selective Search 等算法在细粒度图像中产生物体或物体部位可能出现的候选框（object proposal）。之后类似于 R-CNN 做物体检测的流程，借助细粒度图像中的 object bounding box 和 part annotation 可以训练出三个检测模型（detection model），一个对应细粒度物体级别检测，一个对应物体头部检测，另一个则对应躯干部位检测。然后，对三个检测模型得到的检测框加上位置几何约束，例如，头部和躯干的大体方位、以及位置偏移不能太离谱等。这样便可得到较理想的物体／部位检测结果。接下来将得到的图像块（image patch）作为输入，分别训练一个CNN，则该CNN可以学习到针对该物体／部位的特征。最终将三者的全连接层特征级联（concatenate）作为整张细粒度图像的特征表示。显然，这样的特征表示既包含全部特征（即物体级别特征），又包含具有更强判别性的局部特征（即部位特征：头部特征，躯干特征），因此分类精度较理想。但在Part-based R-CNN中，不仅在训练时需要借助bounding box和part annotation，为了取得满意的分类精度，在测试时甚至还要求测试图像提供bounding box。这便限制了Part-based R-CNN在实际场景中的应用。**

## Abstract  

通过特定对象局部区域细微外观差异得到的局部的语义可以进行细粒度分类。用于姿态标准化的表示的方法已经提出，但由于对象检测的困难，通常在测试时假定边界框标注。我们提出了一个细粒度分类模型，通过利用由下至上区域提议计算的深度卷积特征来克服这些限制。我们的方法学习整体和局部检测器，在它们之间强制学习几何约束，并从姿态标准化表示预测细粒度类别。对 Caltech UCSD 鸟类数据集的实验证实，我们的方法在端到端评估中胜过了最先进的细粒度分类方法，而无需在测试时间使用边界框。  

## 1. Introduction  

由于相关类别中特定区域外观的细微差异，视觉细粒度分类问题可能非常具有挑战性。与常见的识别相比，细粒度分类旨在区分不同品种或物种或产品模型，并且必须以物体姿态为条件进行可靠识别。面部识别是细粒度识别的经典案例，值得注意的是最好的面部识别方法都会去发现面部特征点并从这些位置提取特征。  

因此，定位目标的局部区域对于建立目标之间的对应关系以及避免因为姿势和相机视角变化带来的影响是很重要的。已经有工作研究了基于部件的方法来解决这个问题。局部定位确实是许多姿态归一化表示方法的瓶颈。之前已经使用了 Poselet 和 DPM 方法再局部定位上取得了一定的成功;只有在测试时给出已知的边界框时，方法通常才能得到较好的局部定位。通过开发一种新颖的深度局部检测方案，我们提出了一种端到端的细粒度分类系统，该系统在测试时不需要给定对象边界框，并且可以实现与先前报道的方法相媲美的性能，这些方法需要测试时给出真实标注框来过滤误报的检测结果。  

卷积网络最近在 ImageNet 挑战上的成功激发了将深度卷积特征应用于相关图像分类和检测任务的进一步工作。Girshick 等人通过将 CNN 应用于一组自下而上的候选区域提议，突破了对象检测的性能，与以前的最佳方法相比，PASCAL 检测性能提高了 30％ 以上。OverFeat 提出使用 CNN 进行目标位置的回来来进行检测。但是，利用深度卷积特征的进展并不局限于基本的目标检测。在许多应用中，如细粒度识别，属性识别，姿态估计等，合理的预测需要准确的区域定位。  

特征学习已被用于细粒度识别和属性估计，但受限于局部区域的工程特征。DPD-DeCAF 使用 DeCAF 作为特征描述符，但依靠基于 HOG 的 DPM 进行局部定位。PANDA 学习了特定部分深度卷积网络，其位置取决于基于 HOG 的 poselet 模型。这些模型缺乏 R-CNN 检测的鲁棒性。在这项工作中，我们探索了一种统一的方法，它使用相同的深度卷积表示进行检测以及局部描述。  

我们推测自下而上区域提议方法取得的进展，如 selective search，可以提高使目标及局部检测的性能。正如我们后面显示的那样，Caltech-UCSD 鸟类数据集使用 selective search proposals 的平均召回率为 95％。  

在本文中，我们提出了一个局部定位模型，它通过利用自下而上 selective search proposals 上计算的深度卷积特征来克服先前的细粒度识别系统的局限性。我们的方法学习局部外观模型并强化局部之间的几何约束。我们的方法概述如图 1 所示。我们已经研究了不同的几何约束，包括在语义外观空间中以最近邻为条件的联合区域定位的非参数模型。用广泛使用的细粒度基准 Caltech-UCSD 鸟类数据集来评估我们的方法，并得到 state-of-the-art 结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/1.jpg?raw=true) 

## 2.  Related work  

### 2.1 Part-based models for detection and pose localization  

以前的工作已经提出了对对象局部外观和位置的显式建模，以便更精确地识别和定位。从图像结构开始，通过 poselets 和相关工作，许多方法已经联合定位了一系列的几何相关的区域。可变形零件模型（DPM），直到最近还是 PASCAL 物体检测最先进的方法，在与整个对象边界框相结合的位置上添加附加学习滤波器的模型部分，允许部件以已学会的变形成本对 anchor 进行偏移。DPM 将这种方法用于强监督训练，其中区域位置在训练时进行标注。这些方法的局限性在于它们使用弱特征（通常是 HOG）。  

### 2.2 Fine-grained categorization  

最近，大量的计算机视觉研究集中在许多领域的细粒度分类问题，如动物品种或物种，植物物种和人造物体。  

有几种方法基于检测和提取物体某些部分的特征。Farrell 等人提出了一种使用 poselets 的姿态归一化表示法。可变形零件模型在也被用于局部定位。基于定位脸部基准特征点的工作，Liu等人提出了一种基于样本的几何方法来检测狗脸并从关键点提取高度局部化的图像特征以区分狗的品种。此外，Berg 等人通过学习每对关键点的描述符，学习了一组高度区分的中间特征。此外，还有文章通过强化姿态和子类一致性来扩展以前的非参数样本法。Yao 等人和Yang 等人研究了模板匹配方法以降低滑动窗口方法的成本。 Goring 等人最近的工作将局部标注从具有类似全局形状的对象转换为非参数局部检测。但是，所有这些 part-based 的方法在测试时都需要真实边界框来进行局部定位或关键点预测。  

Human-in-the-loop 的方法要求人们命名对象的属性，点击某些部分或标记最具区分性的区域以提高分类的准确性。基于分割的方法对于细粒度识别也是非常有效的。有方法使用 region-level 线索来推断前景分割 mask 并丢弃背景中的噪声视觉信息。Chai 等表明联合学习局部定位和前景分割可以有益于细粒度分类。与大多数先前的 part-based 的方法类似，这些方法需要真实边界框来初始化分割种子。相比之下，我们的工作目标是在测试时不知道真实标注框的情况下进行端到端的细粒度分类。我们的局部检测器使用基于自底向上区域提议的卷积特征，以及学习的非参数几何约束来更精确地定位对象区域，从而实现强大的细粒度分类。  

### 2.3  Convolutional networks  

近年来，卷积神经网络（CNN）已被纳入各种领域的许多视觉识别系统中。这些模型的强度至少有一部分在于它们能够从原始数据输入（例如图像像素）中学习辨别特征，这与作为初始预处理步骤计算图像上的手工设计特征的更传统的物体识别流水线相反。LeCun 及其同事最初将这些模型应用于数字识别和OCR，后来应用于通用对象识别任务，CNN 得到了推广。随着大型标签图像数据库和 GPU 实现的引入，这些网络已经成为通用对象分类中最准确的方法。  

最近，通用对象检测方法已经开始利用深度 CNN，并且胜过基于传统特性的任何竞争方法。 OverFeat 使用 CNN 在粗滑动窗口检测框架中回归对象位置。对我们工作特别有启发的是 R-CNN 方法，该方法利用区域提案框架中 CNN 的深度特征，在 PASCAL VOC 数据集上实现前所未有的目标检测结果。我们的方法通过将 R-CNN 应用于除了整个对象之外的局部检测来推广 R-CNN，我们的实证结果证明这对精确的细粒度识别是至关重要的。  

## 3.  Part-based R-CNNs  

虽然证明了 R-CNN 方法在通用物体检测任务（PASCAL VOC）上的有效性，但没有探索这种方法同时在定位和细粒度识别中的应用。因为我们的工作是在这个机制中运作的，所以我们扩展了 R-CNN 来检测物体并将它们的局部定位并对定位进行几何约束。关于感兴趣对象的单个语义部分位置的假设（例如，用于动物类的头部位置），对于倾向于出现在相对于这些粗略固定的位置中的微妙外观差异进行建模变得合理部分。  

在 R-CNN 方法中，对于特定对象类别，具有 CNN 特征描述符 $\phi (x)$ 的候选检测 x 被赋予 $w_0^T \phi (x)$，其中 $w_0$ 是对象类别的 SVM 学习向量的权重。在我们的方法中，我们假设一个强有力的监督设置，在训练时间，我们不仅为整个对象设定了真实标注框，而且对一组固定的局部区域也进行了标注 ${p_1,p_2,...,p_n}$。  

在给定这些局部标注的情况下，在训练时，所有对象及其每个区域都被视为独立的对象类别：我们在区域提议中提取的特征描述符上训练一对多线性 SVM，其中与标注框重叠超过 0.7 的 proposal 被标注为该区域的正例，并且与任何标注框区域重叠小于 0.3 的区域被标记为负样本。因此，对于单个对象类别，我们为整体和局部 ${p_1,p_2,...,p_n}$ 分别学习了 SVM 权重 $w_0$ 和 ${w_1,w_2,...,w_n}$。在测试时，对于每个 proposal，我们计算来自所有SVM 的分数。当然，这些分数并不包含任何有关物体及其局部几何约束的知识；例如，在没有任何额外限制的情况下，鸟头检测器可以在检测器检测区域之外被检测到。因此，我们的最终联合对象和局部假设是使用下面几节中详述的几何打分函数来计算的，它强化了姿势预测与训练时观察姿势的统计数据一致的直观理想属性。  

### 3.1 Geometric constraints  

设 $X = {x_0，x_1，...，x_n}$ 表示对象 $p_0$ 和 n 个局部区域 ${p_i}_{i=1}^n$ 的位置（边界框），它们在训练数据中被标注，但在测试时是未知的。我们的目标是在先前未见过的测试图像中推断出物体位置和局部区域位置。给定目标和局部区域的 R-CNN 权重 ${w_0，w_1，...，w_n}$，我们将有相应的检测器 ${d_0，d_1，...，d_n}$，其中每个检测器得分为 $d_i(x)= \sigma (w_i^T \phi (x))$，其中 $\sigma (\cdot)$ 是 sigmoid 函数，$\phi (x)$ 是在位置 x 处提取的 CNN 特征描述符。我们通过解决以下优化问题来推断对象和局部区域的联合配置：  

$X^* = arg \ max_X \ \Delta(X) \prod_{i=0}^n d_i(x_i)$

其中 $\Delta(X)$ 定义了边界框在联合配置上的评分函数。我们考虑并报告了几个配置评分函数 $\Delta$ 的定量结果，在以下段落中详细介绍。  

框约束。 对物体和零件进行局部化的一个直观的想法是考虑每个可能的物体窗口和物体内的所有窗口，并选择具有最高分数的窗口。 在这种情况下，我们定义评分函数  

**边界框约束**。对物体和局部区域进行定位的一个直观的想法是考虑每个可能的物体窗口和物体内的所有窗口，并选择具有最高分数的窗口。在这种情况下，我们定义评分函数  

$\Delta_{box}(X) = \prod_{i=0}^n c_{x_0}(x_i)$

其中  

$ c_x(y) = \begin{cases} 1,&\text{if region y falls outside region x by at most $\epsilon$ pixels} \\ 0,&\text{otherwise} \end{cases} $

在我们的实验中，$\epsilon = 10$。  

**几何约束**。由于单个的区域检测器不完善，单个区域检测器得分最高的窗口并不总是正确的，特别是在有遮挡的情况下。因此，我们考虑几个评分函数，以限制检测结果相对于目标位置的布局从而过滤掉不正确的结果。我们定义  

$\Delta_{geometric}(X) = \Delta_{box}(X) \prod_{i=0}^n \delta_i (x_i)$

其中 $\delta_i$ 是给定训练数据的区域 $p_i$ 的位置的评分函数。在之前关于区域定位的工作中，我们试验了$\delta$的三个定义：  

- $\delta_i^{MG}(x_i)$ 将训练数据中区域 $p_i$ 的 $N_g$ 个成分拟合一个高斯混合模型。实验中，我们设定 $N_g = 4$。  

- $\delta_i^{NP}(x_i)$ 找出 $\tilde{x}_0$ 相似的 K 个最近邻，其中 $\tilde{x}_0 = arg \ max d_0(x_0)$ 是检测器检测结果评分最高的窗口，然后将这 K 个 窗口拟合一个高斯模型。实验中，设置 K = 20。图 2 展示了最近邻的一些例子。  

DPM 用每个分量的高斯先验来模拟 deformation costs。 R-CNN 是一个单成分模型，需要定义$\delta^{MG}$ 或 $\delta^{NP}$。我们的 $\delta^{NP}$ 定义受 Belhumeur 等人的启发，但不同之处在于我们在外观上而不是几何上定义最近邻。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/2.jpg?raw=true)

### 3.2 Fine-grained categorization  

我们从局部区域以及整个对象中提取语义特征。最终的特征表示是 $ [\phi (x_0),...,\phi (x_n)] $ 其中 $x_0$ 和 $x_{1...n}$ 是使用上一节中的模型推导的整体和局部检测结果，$ \phi (x_i) $ 是区域 $x_i$ 的特征表示。  

在一组实验中，我们从 ImageNet 预先训练的 CNN 中提取深卷积特征 $ \phi (x_i) $，类似于 DeCAF。为了使 CNN 特征在细粒度鸟类分类的目标任务具有更强去区分性，我们还将从 CUB 数据集根基标注框切分的 200 类鸟类分类任务对 预训练的 CNN 网络进行 fune-tune。特别是，我们用一个新的 200 路 fc8 分层替换了原始的 1000 路 fc8 分类层，并随机初始化这层的权重。我们根据 R-CNN 的建议设置微调的学习率，将全局速率初始化为初始ImageNet 学习速率的十分之一，并在整个训练过程中将其降低 10 倍，但在新的 fc8 中学习速率这是全局学习率的 10 倍。对于整个对象边界框和每个区域边界框，我们独立地对 ImageNet 预先训练的 CNN 进行微调，以对每个区域进行分类，对每个区域进行调整为 227×227 网络输入大小。测试时，我们使用针对特定整个对象或局部区域进行微调的网络，为预测的整个对象或局部区域提取特征。  

为了对分类器进行训练，我们采用了一个一对多的线性 SVM 来使用最终的特征表示。对于新的测试图像，我们将全部和局部检测器与几何打分函数一起应用，以获取检测到的局部区域位置并使用特征进行预测。如果在测试图像中没有检测到特定区域 i（由于所有提议低于局部检测器的阈值），我们设置其特征 $ \phi (x_i) = 0$（零矢量）。  

## 4. Evaluation  

在本节中，我们对提出的方法进行了评估。具体而言，我们在广泛使用的细粒度基准 Caltech-UCSD 鸟类数据集上进行了实验。分类任务是区分 200 种鸟类，并且由于类别之间的高度相似性从而对计算机视觉系统具有挑战性。它包含 200 种鸟类的 11,788 幅图像。每个图像都用边界框和 15 个关键点的图像坐标标注：喙，背部，胸部，腹部，额头，冠，左眼，左腿，左翼，右眼，右腿，右翼，尾巴，颈背 和喉咙。我们对包含在数据集中的分组进行训练和测试，其中每个物种约 30 个训练样本。遵循之前的协议，我们使用鸟类数据集的两个语义部分：头部和身体。  

我们使用开源软件包 Caffe 来提取深度特征并微调我们的 CNN。对于目标和局部区域检测，我们使用 Caffe 参考模型，它与 Krizhevsky 等人使用的模型几乎相同。我们将 CNN 第 n 层的深层特征称为 convn，pooln 或 fcn，分别是卷积层，汇聚层或完全连接层的输出。我们使用 fc6 来训练 R-CNN 目标和局部区域检测器以及用于分类的图像表示。对于 $\delta^{NP}$，使用 pool5 和余弦距离度量来计算最近邻。  

### 4.1 Fine-grained categorization  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/3.jpg?raw=true)

我们首先展示 Caltech-UCSD 鸟类数据集的标准细粒度分类任务上的结果。表 1 中的第一组结果是在测试时整个鸟类的真实标注框已知的情况下实现的，因为大多数 state-of-art 方法假设这样会使分类任务更容易一些。在这种情况下，我们具有局部非参数几何约束 $\delta^{NP}$ 的 part-based 方法在没有微调的情况下效果最好，无需微调即可实现 68.1％ 的分类精度。 Finetuning 大幅改善了这一结果，达到 76％ 以上。我们将结果与三种最先进的基准方法进行比较。我们使用深度卷积特征，但他们使用基于 HOG 的 DPM 作为其局部定位方法。性能的提高可能是由于更好的局部区域定位（见表4）。Oracle 方法在训练和测试时都使用真实的整体和局部标注。  

第二组是在测试时未知鸟的标注框的结果。大多数的文章都没有展示在这个更真实、更复杂数据集上的效果。如表 1 所示，在此设置中，我们 part-based 的方法比基准 DPD 模型效果更好。我们在不进行微调的情况下获得 66.0％ 的分类精度，几乎与给出真实边界框时的精度一样好。这意味着在测试时不需要对任何目标进行标注就可以进行分类。通过微调 CNN 模型，我们的方法分类准确率达到 73.89％。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/4.jpg?raw=true)

我们做的另一个有趣的实验是移除局部区域描述符仅使用预测框中的整体图像描述符。通过几何约束，我们的方法能够帮助局部区域定位。如表 2 所示，我们的方法优于使用 R-CNN 的单个对象检测器，这意味着几何约束有助于我们的方法更好地定位对象窗口。强 DPM 的检测不如我们的方法准确，这就解释了它性能下降的原因。oracle 方法使用真实边界框，并且达到了 57.94％ 的准确率，这仍然远低于表 1 中在对象和局部区域内使用两个图像描述符的方法。  

### 4.2 Part localization  

我们现在展示我们系统在定位局部区域上的效果。在表 4 中的结果是根据正确定位的区域分（PCP）度量的百分比给出的。对于第一组结果，给出了整个对象边界框，任务仅仅是定位边界框中局部区域的位置，与真实局部区域标注相交面积大于 0.5 的认为是正确的结果。  

对于第二组结果，使用在第二部分中描述的目标函数在排名最高的局部区域预测上计算 PCP 度量。请注意，在这个更真实的设置中，我们不会假设在测试时就知道真实的目标标注框。  

如表 4 所示，对于给定边界框和未知边界框的两种设置，我们的方法都优于强 DPM 方法。增加一个几何约束 $\delta^{NP}$ 可以改善我们的结果。在全自动设置中，头部排名最高的检测和局部区域定位性能比基准方法好 65％。 $\Delta_{null}= 1$是没有应用几何约束的仅外观情况。虽然细粒度的分类结果没有显示 Δgeometric 和 Δbox 之间的很大差距，但我们可以看到局部区域定位性能的差距。性能差距较小的原因可能在于深层卷积特征对于小的变化和旋转是不变的。  

我们还评估了对边界框和局部区域 selective search region proposals 的 recall。表 3 列出了不同重叠阈值的召回结果。重叠要求为 0.5 时，鸟头部和身体部位的召回率很高，这为根据 region proposals 定位这些部件提供了基础。但是，我们还观察到，当重叠阈值为 0.7 时，头部召回率低于 40％，这表明自下而上的 region proposals 可能是精确局部定位的瓶颈。  

其他可视化显示在图 4 中。我们显示了每个图像的三个检测和局部区域定位，第一列是强 DPM 的输出，第二列是我们的单个局部区域预测的方法，最后一列是我们使用局部优先的方法。我们使用预训练的模型来获得结果。我们还会在图 5 中显示我们方法的一些失败案例。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/5.jpg?raw=true)

### 4.3 Component Analysis  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/6.jpg?raw=true)

为了检验 $\Delta_{geometric}$ 中使用的 $\alpha$ 和 K 不同值的影响，我们进行交叉验证实验。结果显示在图 3 中，我们将图 3 中的 K = 20 固定，并将图 3 中 的 $\alpha = 0.1$ 固定，所有关于以交叉验证方式训练数据的实验，我们将训练数据分成 5 次。如结果所示，端到端的细粒度分类结果对 $\alpha$ 的选择非常敏感，并且 $\alpha = 0.1$ 是没有任何几何约束的 $\Delta_{box}$ 预测的情况。我们必须选择一个小 $\alpha$ 的原因是高斯的 pdf 与我们的局部区域检测器输出的 logistic score 相比较大。另一方面，K 的选择不能太小，当 K 大于 10 时它的选择不是很敏感。  

## 5. Conclusion  

我们已经提出了一种能够进行最先进的细粒度目标识别的联合目标检测和区域定位系统。我们的方法学习探测器和区域模型，并强化区域之间和对象框架之间的几何约束。我们的实验结果表明，即使具有非常强大的特征表示和对象检测系统，通过区域来为具有高语义相似度的类别之间的细粒度区分的困难任务额外地建模对象的姿态也是非常有益的。在今后的工作中，我们将考虑在训练时联合建模对象类别及其各个部分和变形成本的方法。我们还计划探索弱监督设置，在该设置中，我们只会自动发现局部区域并将其模型化为仅来自对象边界框标注的潜在变量。最后，我们将考虑放宽对较小部分的 selective
search，并采用密集窗口采样。

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/7.jpg?raw=true)

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/8.jpg?raw=true)

# Mask-CNN: Localizing Parts and Selecting Descriptors for Fine-Grained Image Recognition  


## Abstract  

细粒度的图像识别是一个具有挑战性的计算机视觉问题，因为由高度相似的从属类别引起的小类间变化，以及姿势，尺度和旋转引起的大的类内变化。本文中，我们对细粒度识别提出了一种新的没有全连接层的端到端 Mask-CNN 模型。基于细粒度图像的局部区域标注，所提出的模型由完全卷积网络组成，以定位可区分的部分（例如，头部和躯干），并且更重要的是生成用于选择有用和有意义的卷积描述符的目标/局部掩码。之后，建立一个 Mask-CNN 模型，用于同时聚合选定的对象和局部区域级描述符。与现有细粒度方法相比，所提出的 Mask-CNN 模型具有最少的参数，最低的特征维度和最高的识别准确度。  

## 1. Introduction  

细粒度的识别任务，例如识别鸟的种类，在计算机视觉中已经很流行。由于类别彼此相似，因此不同的类别只能通过轻微细微的差异来区分，这使得细粒度识别成为具有挑战性的问题。与一般的对象识别任务相比，细粒度识别更有益于学习对象的关键部分，这些部分这有助于区分不同的子类并对齐同一类的数据。  

在深度学习时代，区域表示的直接方式是使用深度卷积特征/描述符。与全连接层的特征（即，整个图像）相比，卷积描述符包含更多的局部（即，局部区域）信息。另外，已知这些深度描述符对应于中级信息，例如目标局部区域。所有先前 part-based 的细粒度方法，都直接使用深卷积描述符并将它们编码成单一表示，而不评估获得的对象/局部区域深度描述符的有用性。通过使用强大的卷积神经网络，我们可能不需要在特征向量内选择有用的维度，就像我们对手工特征所做的那样。但是，由于大多数深度描述符对细粒度识别没有用处或意义，因此有必要选择有用的深度卷积描述符。最近，选择深度描述符对细粒度图像检索任务进行了阐述。而且，这对细粒度图像识别也是有益的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/1.jpg?raw=true)

在本文中，通过开发一种新颖的深度局部区域检测和描述符选择方案，我们提出了一种端到端的 Mask-CNN（M-CNN）模型，该模型丢弃全连接的层以进行细粒度识别。在训练期间，我们只需要局部区域标注和图像级标签。在 M-CNN 中，给定局部区域标注，我们首先将它们分成两个点集。一组对应于细粒鸟图像的头部部分，另一组对应于躯干。然后，将覆盖每个点集的最小凸多边形作为 ground-truth mask 返回，如图 1 所示。其他像素为背景。通过将局部定位视为三类分割任务，我们充分利用全卷积网络（FCN）在测试时生成 mask 并选择有效的深度描述符，在测试期间不使用任何标注。获得这两部分 mask 后，我们将它们组合起来形成对象。基于这些对象/局部区域 mask，构建了一个四流 four-stream Mask-CNN（图像，头部，躯干，对象），用于联合训练并同时聚合对象级和局部区域级特征。所提出的 four-stream M-CNN 的架构如图 2 所示。在每个 M-CNN 流中，我们丢弃全连接的 CNN 层。在最后的卷积层中，输入图像由多个深度描述符表示。为了选择有用的描述符以仅保留与对象相对应的描述符，使用 FCN 预先学习的对象/局部 mask。之后，每个流选定的描述符都被 averaged pooled 和 max pooled 成 512 维的向量，然后使用 $l_2$ 标准化。最后，将这四个流的特征向量连接起来，然后添加一个分类（fc + softmax）层进行端到端联合训练。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/2.jpg?raw=true)

我们在 Caltech-UCSD Birds-200 2011 数据集上验证了提出的四流 M-CNN，我们实现了 85.5％ 的分类精度。也获得准确的局部区域定位（头部 84.62％和躯干 89.83％）。所提出的 M-CNN 模型的主要优点和贡献是：  

- 据我们所知，Mask-CNN 是第一个选择深卷积描述符用于目标识别的端对端模型，特别是对于细粒度图像识别。  

- 我们提出了一种用于细粒度识别的新颖且高效的 part-based four-stream 模型。丢弃全连接层，并且所提出的 M-CNN 在计算和存储方面是有效的。 与最先进的方法相比，M-CNN 的参数最小，特征维数最小（60.49M和8192-d）。同时，CUB200-2011 的分类准确率达到 85.4％，是现有方法中最高的。使用 SVD 白化方法，我们的特征表示可以压缩到 4,096-d，同时将精度提高到 85.5％。  

- 该模型的局部区域定位性能优于其他需要额外标注框的 part-based 的细粒度方法。特别是，M-CNN 比用于头部定位的最新技术高出约 10％。  

## 2. Related Work  

细粒度识别是一个具有挑战性的问题，最近已成为一个热门话题。在过去的几年中，文献中已经发展了许多有效的细粒度识别方法。我们可以粗略地将这些方法分为三组。第一组试图通过开发用于分类细粒度图像的强大深度模型来学习更具辨别性的特征表示。第二组对细粒度图像汇总的目标进行矫正，以消除姿势变化和相机位置的影响。最后一组关注于 局部区域的表示，因为人们普遍认为细粒度图像之间的细微差别大多存在于对象局部区域的的独特属性中。  

对于 part-based 的细粒度识别方法，有论文在训练期间使用鸟类的边界框和局部区域标注来学习准确的局部区域定位模型。然后，分别使用检测到的部分对不同的 CNN 进行微调。为了确保令人满意的定位结果，他们甚至在测试阶段使用了边界框。相比之下，我们的方法只需要在训练时使用局部区域标注，并且在测试过程中不需要任何标注。此外，我们的四流 M-CNN 是同时捕获对象和局部区域级信息的统一框架。其他一些 part-based 的方法被认为是一种弱监督的设置，在这种设置中，他们将细粒度图像仅分为图像级标签。正如我们的实验所显示的那样，M-CNN 的分类精确度明显高于这些弱监督方法。同时，M-CNN 的模型尺寸在所有最先进的方法中是最小的，这使得训练效率更高。  

此外，还有基于分割的细粒度识别方法。它们与 M-CNN 最显著的区别在于：这些方法仅使用分割来定位整个对象或局部区域，而我们使用分割中的 mask 进一步选择有用的深度卷积描述符。其中，part-stacked 的 CNN 模型是与我们最相关的工作。part-stacked CNN 在训练中需要目标标注框和局部区域标注，甚至在测试期间也需要目标标注框。在边界框裁剪的图像块内，该方法将 15 个关键点中的每个关键点周围的图像作为 15 个分割前景类别，并使用FCN 解决 16 类分割任务。在获得训练的 FCN 之后，它将这些局部点位置定位在最后的卷积层中。然后，将对应于十五个区域的激活和整个对象堆叠在一起，全连接层被用于分类。与 part-stacked CNN 相比，M-CNN 只需要定位两个主要部分（头部和躯干），这使得分割问题更容易和更准确。如表 3 所示，M-CNN 实现了很高的定位精度。同时，part-stacked CNN 使用全部 15 个局部定位不会导致更好的分类准确率。此外，虽然我们在训练中使用较少的标注，并且在测试中不使用任何标注（参见4.2.2节），但 MUBN 的 CUB200-2011 的准确率比使用相同 baseline network 的 part-stacked CNN 高1.8％ 。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/3.jpg?raw=true)

## 3. The Mask-CNN Model  

在本节中，我们展示了提出的四流 Mask-CNN（M-CNN）模型。首先，我们采用全卷积网络（FCN）来生成用于定位对象/局部的对象/局部掩模，并且更重要的是选择深度描述符。然后，基于这些 mask，建立四流 M-CNN，用于联合训练并捕获对象和局部区域级别的信息。  

### 3.1 Learning Object and Part Masks  

全卷积网络（FCN）设计用于像素级别分类。FCN 可以以任何分辨率拍摄输入图像并产生相应尺寸的输出。在我们的方法中，我们使用 FCN 不仅将细粒度图像中的对象和局部区域进行定位，而且还将分割预测作为后面的描述符选择过程的 masks。  

CUB200-2011 数据集中的每个细粒图像都对局部区域进行了标注，例如 15 个关键点。如图 1 所示，我们将这些关键点分成两组，包括头部关键点（即喙，额头，冠，左眼，右眼，颈背和喉咙）和躯干关键点（即背部，乳房，腹部，左腿，右腿，左翼，颈背，右翼，尾巴和喉咙）。基于关键点，生成了两个局部标注 mask。一个是头部 mask，它对应于覆盖所有头部关键点的最小凸多边形。另一个是躯干 mask，它是覆盖躯干关键点的最小凸多边形。在图 1中，红色多边形是头部 mask，蓝色是躯干 mask。图像的其余部分是背景。因此，我们将 part mask 学习过程建模为三类分割问题。为了进行有效的训练，所有训练和测试的细粒度图像均使用其原始分辨率。然后，我们在原始图像的中间裁剪一个 384×384 的图像块作为输入。mask 学习网络结构如图 3 所示。在我们的实验中，我们采用了 FCN-8s 来学习和预测 part masks。  

在 FCN 推断期间，不使用任何标注，为每个图像返回三个类别热图（与原始输入图像大小相同）。我们随机选择一些预测的 part masks，并在图 4 中显示它们。在这些图中，学习的 mask 覆盖在原始图像上。头部以红色突出显示，躯干呈蓝色。预测的背景像素为黑色。从这些图中可以看出，即使真实标注的局部 mask 不是非常准确，学习的 FCN 模型也能够返回更准确的 part mask。同时，这些局部 mask 也可以定位局部位置。局部定位和对象分割的定量结果将在后面章节报告。  

如果两局部 mask 都精确预测，将有利于后面的深度描述符选择过程和最终的细粒度分类。 因此，在训练和测试期间，我们将使用 M-CNN 预测的 masks 进行局部定位和描述符的选择。我们还将两个 mask 组合在一起，为整个对象形成一个 mask，称之为对象 mask。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/4.jpg?raw=true)

### 3.2 Training Mask-CNN  

在获得对象和局部 mask 模后，我们构建了四流 M-CNN 进行联合训练。该模型的整体结构如图 2 所示。我们以整个图像流为例来说明 M-CNN 中每个流的流水线。  

整个图像流的输入是用 h×h 调整大小的原始图像。在我们的实验中，我们分别报告了 h = 224 和 h = 448 的结果。输入图像被馈送到传统的卷积神经网络中，但全连接层被丢弃。也就是说，我们提出的 M-CNN 中使用的 CNN 模型仅包含卷积，ReLU和汇聚层，这大大降低了 M-CNN 模型的大小。具体而言，我们使用 VGG-16 作为基准模型，并保留 pool5 之前的层（包括pool5）。如果输入图像为 224×224，我们在 pool5 中获得 7×7×512 激活张量。因此，我们有 49 个 512-d 的深度卷积描述符，它们也对应于输入图像中的 7×7 个空间位置。然后，学习对象 mask（参见3.1节）首先通过最近的插值重新调整为7×7，然后用于选择有用和有意义的深度描述符。如图 2（c）和（d）所示，描述符在位于目标区域时应保持不变。如果它位于背景区域中，则该描述符将被丢弃。在我们的实现中，mask 设置为二进制矩阵，其中 1 表示保留，0 表示丢弃。我们将选择过程作为卷积激活张量与 mask 矩阵之间的元素内积作来实现，这与 FCN 中的元素相加操作相似。因此，位于目标区域中的描述符将保留，而其他描述符将成为零向量。  

对于这些选定的描述符，在端到端的 M-CNN 学习过程中，我们将它们分别平均和最大化为两个 512-d 特征向量。然后，对他们中的每一个都进行 $l_2$ 归一化。之后，我们将它们连接成 1024-d 特征作为整个图像流的最终表示。  

用于头部和躯干的流与整个图像具有相似的处理步骤。然而，与整个图像流的输入不同，我们如下生成头部和躯干流的输入图像。在获得两部分 mask（即头部和躯干掩模）之后，我们使用 part masks 作为 part detectors 来定位输入图像中的头部部分和躯干部分。对于每个 part，我们返回包含 part masks 区域的最小矩形边界框。基于矩形边界框，我们裁剪作为 part 流输入的图像补丁。图 2 中间显示了 M-CNN 的头部和躯干流。最后一个流是对象流，它通过将两部分掩码组合到对象掩码中来裁剪图像补丁。因此，它的输入是我们的 FCN 分割网络检测到的主要对象（即，鸟）。在我们的实验中，这三个流的输入都调整为224×224。  

在图 2（f）所示的分类步骤中，最终的 4096d 图像表示是整个图像，头部，躯干和对象特征的拼接。M-CNN 的最后一层是用于 CUB200-2011 数据集分类的 200 路分类（fc + softmax）层。 四个流 M-CNN 是端到端学习的，同时学习四个 CNN 的参数。在训练 M-CNN 期间，所学习的 FCN 分割网络的参数是固定的。  

## 4 Experiments  

在本节中，我们首先描述实验设置和实现细节。然后，我们报告分类的准确性，并对所提议的 M-CNN 模型进行讨论。最后，还将提供 part 定位和对象分割的性能。  

### 4.1 Dataset and Implementation Details  

是在广泛使用的细粒度基准鸟类数据集 Caltech-UCSD 2011 上进行的评估的。该数据集包含 200 种鸟类，每个类别包含大约 30 个训练图像。在训练阶段，采用 15 个局部标注生成 part masks，同时将图像级标签用于端到端的 M-CNN 联合训练。测试时，我们不需要监督信号（例如，局部标注或边界框）。  

所提出的 Mask-CNN 模型和用于生成掩码的 FCN 使用开源库 MatConvNet 来实现。在我们的实验中，在获得学习到的 mask 后，我们首先生成鸟头，躯干和物体的图像块，如第二节所述。然后，为了促进四流 CNN 的收敛，对应于整个图像，头部，躯干和物体的每个单独的流在其输入图像上被分别微调。每个流中使用的 CNN 由在 ImageNet 上预先训练过的 VGG-16 模型初始化。另外，我们通过对所有四个流进行水平翻转来使训练数据翻倍。如图 2 所示，在对每个流进行微调之后，执行四流 M-CNN 的联合训练。在 M-CNN 中不使用 dropout。在测试时，我们对图像及其翻转副本的预测进行平均，并输出具有最高分数的类作为测试图像的预测。此外，与逻辑回归（LR）相比，直接使用 softmax 预测结果的准确性略有下降，这与之前中的观察结果一致。因此，在下文中，所报告的 M-CNN 结果全部是用提取的特征（4096-d）与默认超参数 $C_LR = 1$ 基于 one-vs-all logistic regression 实现的。  

### 4.2 Classification Accuracy and Comparisons  

我们报告了提出的四流 M-CNN 模型在 CUB200-2011 数据集上的分类准确率，并与文献中的基线方法和最新方法进行比较。  

#### 4.2.1 Baseline Methods  

为了验证 M-CNN 中描述符选择过程的有效性，我们执行了基于所提出的四流体系结构的两种基线方法。与我们的 M-CNN 不同，这两个基线方法不包含描述符选择部分，即图 2（d）所示的处理。  

第一种基线方法使用传统的全连接层来对每个流进行分类，其被称为“4 流 FC”。 在“4 流 FC”中，我们用含有全连接层的 CNN（即，仅移除了 fc8 的VGG-16）代替图 2 中每个流的（b）到（e）部分。 因此，每个流的最后一层中生成的特征是一个 4096-d 单个矢量。 其余过程也是将 4 个 4,096d 特征连接到最后一个特征 16,384-d，并在 16,384d 图像表示上学习 200 路分类（fc + softmax）层。  

第二个基线与提议的 M-CNN 类似。最显着的区别是它丢弃了描述符选择部分，即图 2（d）中的处理。因此，每个流中 pool5 的卷积深度描述符都是直接平均和最大值汇集，然后分别进行 $l_2$ 归一化。因此，我们称之为“4-stream Pooling”。其余程序与提出的 M-CNN 相同。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/5.jpg?raw=true)

#### 4.2.2 Comparisons with state-of-the-art methods  

表 2 列出了提议的四流 M-CNN 和 CUB200-2011 最先进方法的分类准确率。为了公平比较，我们只报告在测试中不使用局部标注的结果。  

如前所述，当所有输入尺寸为 224×224 时，所提出的四流 M-CNN 模型的准确率为 83.1％。之后，我们将整个图像流的输入图像更改为 448×448 像素，这将分类性能提高了 2.1％。我们还将对象流的输入图像大小调整为 448×448。但精度略低于以前。  

此外，由于多层的集合可以提高最终性能，所以在联合训练之后，我们从 pool5 前面三层的 relu5_2 层中提取深层描述符。然后，预测的 part mask 也用于选择四个流的相应描述符。完成与 pool5 相同拼接过程，我们可以获得 relu5_2 的另一个 4,096-d 图像表示形式。之后，我们将它与 pool5 结合成一个 8,192-d 特征向量（表2中称为“4-stream M-CNN +”），在 CUB200-2011 上实现了 85.4％ 的最佳分类精度。 另外，我们通过 SVD 白化将 8,192-d 特征向量压缩到 4,096。它可以降低维度，同时将精度提高到 85.5％。  

特别的，由于 part-stacked CNN 使用了 Alex-Net 模型，我们还基于 Alex-Net 构建了另一个四流 M-CNN。我们的四流 M-CNN（Alex-Net）的准确率为78.0％。它比 part-stacked CNN 高1.8％。此外，在基于 Alex-Net 的四流 M-CNN 中，参数数量仅为 9.74M，最终的特征向量仅为 2,048 维。  

### 4.3 Part Localization Results  

除了在第二节中显示的定位结果外。在本节中，我们使用 Percentage of Correctly Localized Parts（PCP）度量来定量评估定位的准确性。如表 3 中所报告的，度量指标是定位的部分（即头部和躯干）与标准标注IOU超过 50% 的百分比。  

通过比较躯干 PCP 的结果，我们的方法大大优于 part-based R-CNN 和强 DPM。但是，由于我们在测试中不使用任何监督，所以定位性能低于在测试期间使用边界框的 Deep LAC。另外，对于比躯干更具挑战性的头部定位任务，尽管我们的方法仅使用了局部注释来训练，但头部定位性能（84.62％）仍然明显高于其他方法。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/6.jpg?raw=true)

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/2/7.jpg?raw=true)

### 4.4 Object Segmentation Performance  

由于 CUB200-2011 数据集也提供了对象分割标注，因此我们可以直接在分割上测试学习到的 mask。图 5 显示了定性分割结果。我们基于 FCN 的方法通常能够很好地对前景物体进行分割，但可以理解的是，分割更精细的鸟类部分，例如爪和喙是很麻烦的。由于我们的目标不是分割对象，因此我们不会将其作为预处理或后处理进行细化。此外，我们通过真实前景对象与预测对象 mask 的常见语义分割度量均值 IU（像素精度和区域相交联合）来定量评估分割性能。测试集合为 72.41％。实际上，更好的分割结果将导致更好的预测对象/局部 mask，并且也有利于最终的分类。为了进一步提高分类精度，一些预处理方法，例如 GrabCut，值得试着去获得比图 3（c）中的凸多边形更好的 mask。  

## 5 Conclusion  

在本文中，我们介绍了在目标识别中选择深度卷积描述符的好处，特别是细粒度图像识别。通过开发描述符选择方案，我们提出了一种不具有全连接层的新型端到端 Mask-CNN（M-CNN）模型，不仅能够精确地定位对象/局部，还能生成用于选择深读卷积的对象/局部 mask 描述。在聚合选定的描述符后，对象级和部分级特征由所提出的四流 M-CNN 模型编码。Mask-CNN 不仅在 CUB200-2011 上获得了 85.5％ 的分类精度，而且参数参数最少，维度特征表示最低。  

今后，我们计划在弱监督环境中解决 M-CNN 的部分检测问题，其中我们只需要图像级标签。因此，要达到可比较的分类精确度，需要少得多的标注工作量。另外，另一个有趣的方向是探索用于一般对象分类的描述符选择的好处。

# The Application of Two-level Attention Models in Deep Convolutional Neural Network for Fine-grained Image Classification  

## Abstract  

细粒度分类是具有挑战性的，因为类别只能通过细微局部的差异来区分，位置，大小或者旋转都会使问题变得更加困难。大多数细粒度分类系统遵循寻找前景对象或对象局部区域来提取判别特征。在本文中，我们建议将视觉注意力应用于使用深度神经网络的细粒度分类任务。我们集成了三种注意力模型：提出候选 patches 的自下而上的注意力，选择特定目标相关 patches 的目标级别自顶向下的注意力，定位可区分局部区域的局部级别自顶向下的注意力。我们将这些注意力结合起来，对特定领域的深层网络进行训练。重要的是，我们避免使用昂贵的标注，如边界框或局部信息标注。弱监督约束使我们的工作更容易推广。我们已经验证了该方法对 ILSVRC2012 数据集和 CUB200 2011 数据集子集的有效性。我们的 pipeline 在最弱的监督条件下进行了重大改进并取得了最佳的准确性。该性能与其他依赖附加标注的方法相比具有竞争性。  

## 1. Introduction  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/1.jpg?raw=true)

细粒度分类是指在一些基本层次类别下的次分类问题，例如，对不同的鸟类，狗品种，花种，飞机模型等进行分类。这是一个具有广泛应用的重要问题。 即使在 ILSVRC2012 1K 类别中，犬类和鸟类下分别有 118 个和 59 个子类。直观地，如图 1 所示类内差异可能大于类间差异。因此，细粒度分类在技术上具有挑战性。  

具体而言，细粒度分类的困难来源于可区分特征不仅仅是前景中的物体了，更应该是物体的局部信息，比如说鸟的头部。因此，大多数细粒度分类系统都遵循以下流程：找到前景对象或对象局部区域以提取可区分特征。  

为了这个工作，自下而上的过程是不可避免的，我们需要提出很多图像区域作为物体候选者，或者这些区域包含那些具有判别力的部分。selective search 是一个无监督的过程，可以提出数千个这样的区域。这个方法在最近的研究中被广泛使用，我们也采用了这一点。  

自下而上的过程具有较高的召回率，但精度很低。如果对象相对较小，则大多数 patch 都是背景，对对象分类没有帮助。这给 pipeline 中的定位部分带来了问题，导致需要自顶向下的注意力模型来过滤掉嘈杂的 patch 并选择相关的 patch。在细粒度分类的背景下，查找前景对象和对象局部区域可以被看作是一个两层的注意力过程，一个在对象层面，另一个在局部区域层面。  

现有大多数的方法依靠强大的监督来处理注意力问题。他们严重依赖于人为的标签，使用边界框来表示对象级别和局部级别的区域标注。最强大的监督设置既可以在训练中也可以在测试阶段使用，而最弱的设置在训练和测试中都不使用这两种标注。大部分工作介于两者之间。  

由于标注价格昂贵且不可扩展，因此本研究的重点是使用最弱的监督。认识到粒度差异，我们采用两条独立的 pipeline 来实现对象级和部分级的注意力，但共享组件。以下是我们方法的总结：  

- 我们将在 ILSVRC2012 1K 预先训练的卷积神经网络转变成一个 FilterNet。FilterNet 可以选择跟基准类别很相近的 patches，因此可以处理 object-level attention。选定的 patches 用来训练另外一个 CNN，训练成一个 domain 分类器，称为 DomainNet。  

- 经验上，我们观察到了在 DomainNet 中隐藏的聚类模式。多组神经元对可区分部分表现出高度敏感性。因此，我们选择相应的滤波器作为部分检测器来实现 part-level attention。  

在这两个步骤中，我们只需要图像级标签。  

接下来的关键步骤是从这两个注意力模型选择的区域中提取可区分性特征。最近，有令人信服的证据表明，由 CNN 产生的特征可以提供超过手工特征的优越性能。遵循上面概述的两个注意力 pipeline，我们采用相同的策略。在对象级别上，DomainNet 直接输出由图像的多个相关 patch 驱动的 multi-view 预测。在 part-level 上，由被检测部件驱动的 CNN 隐藏层的激活通过基于部件的分类器产生另一个预测。以利用两级注意力的优势，最终的分类合并了两条 pipeline 的结果。  

我们的初步结果证明了这种设计的有效性。使用最弱监督，我们改进了 ILSVRC2012 数据集中犬类和鸟类的细粒度分类，错误率从 40.1% 和 21.1% 降到了 28.1% 和 11%。在 CUB200-2011 数据集中，我们达到了 69.7％ 的准确性，与其他使用更强监督的方法相比具有竞争力。我们的技术通过更好的网络会有改善，例如使用 VGGNet 的精度达到近 78％。  

本文的其余部分安排如下。我们首先在第 2 节中描述利用对象级和部分级注意力进行细粒度分类的 pipeline。详细的性能研究和分析将在第 3 节中介绍。相关工作在第 4 节中介绍。最后，讨论我们学到的东西和未来的工作。  

## 2. Methods  

我们的设计基于一个非常简单的直觉：执行细粒度分类首先需要“看到”物体，然后才能看到物体中最具有区别性的部分。在图像中寻找奇瓦瓦狗需要首先看到一只狗，然后专注于它的重要特征，以区别于其他品种的狗。  

为了实现这一点，我们的分类器不应该在原始图像上工作，而应该在指定的 patch 上工作。这些 patch 还应该保留与识别步骤相关的可区分行。在上面的例子中，第一步的目标是在狗类分类阶段，第二阶段的目标是在奇瓦瓦狗与其他品种（例如耳朵，头部，尾巴）之间进行区分的部分。至关重要的是，认识到详细标签价格昂贵且难以扩展的事实，我们选择使用最薄弱的标签。具体来说，我们的 pipeline 只使用图像级标签。  

原始候选 patch 是在自底向上的过程中生成的，将像素分组为几个区域，突出显示某些对象局部区域。在这个过程中，我们采用 selective search 从输入图像中提取 patch。这一步将提供原始图像的多尺度和多角度图像。但是，自下而上的方法将提供高召回率但低精度的 patch。需要应用自顶向下的注意力来选择对分类有用的相关 patch。  

### 2.1. Object-Level Attention Model  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/2.jpg?raw=true)

**使用目标级别 attention 选择 patch** 此步骤通过自上而下的目标级 attention 过滤自下而上的原始 patches。目标是消除与对象无关的噪声patch，这对训练分类器十分重要。通过将在 ILSVR2012 数据集 1k 个类上训练的 CNN 转换为目标级 FilterNet 来实现这一点。我们将细粒度图像父类（例如奇瓦瓦父级是狗）softmax 的激活值作为选择 patch 的置信度，然后在分数上设置阈值以决定是否选择给定的 patch。这在图 2 中示出。通过这种方式，多尺度和多视图的优势已被保留，并且噪声也被滤除了。  

**训练 DomainNet** 由 FilterNet 选择的 patch 用于训练适当变形后新的 CNN。我们将这个第二 CNN 称为 DomainNet，因为它提取了与属于特定类别（例如，狗，猫，鸟）相关的特征。  

我们注意到，从单个图像中可以得到很多这样的 patch，并且网络的作用就像是数据增强的集合。与其他数据增强（如随机裁剪）不同，我们对 patch 与目标之间存在更高的相关性。数据量也推动了对更大网络的训练，使其能够构建更多特征。这有两个好处。首先，DomainNet 本身就是一个很好的细粒度分类器。其次，它的内部特征现在允许我们构建局部探测器，我们将在下面进行解释。  

**目标级别 attention 分类** 通过目标级 attention 选择的 patches 可以自然地应用于测试阶段。为了获得图像的预测标签，我们向 DomainNet 输入由 FilterNet 选择的 patches。然后计算所有 patches 的 softmax 输出的平均分类分布。最后，我们可以得到平均 softmax 分布的预测。  

该方法包含超参数置信度阈值，它会影响选定 patches 的质量和数量。实验中，我们将其设置为 0.9，因为此值提供了最佳的验证准确度和可容忍的训练时间。

### 2.2. Part-Level Attention Model  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/3.jpg?raw=true)

**构建局部检测器** DPD 和 Part-RCNN 的工作强烈表明，某些局部区域的区分性特征（例如头部和身体）对于细粒度分类至关重要。正如许多相关工作那样，我们不是在局部区域和关键点上使用强标签，而是从 DomainNet 的隐藏层具有聚类特征中受到启发。例如，有多组神经元对鸟头作出反应，有些则对鸟体作出反应，尽管它们可能具有不同的姿势。事后看来，这并不令人意外，因为这些特征确实可以代表一个类别。  

图 3 从概念上展示了这一过程。本质上，我们对相似度矩阵 S 进行谱聚类，将中间层的滤波器分为 k 个组，其中 $S(i,j)$ 表示 DomainNet 中两个中间层滤波器 $F_i$ 和 $F_j$ 权重的余弦相似度。实验中，我们的网络与 AlexNet 基本相同，我们从第四个卷积层挑选神经元，其中 k 设为 3.每组都可以作为一个局部检测器。  

使用聚类滤波器来检测 region proposals 的局部区域的步骤如下：1）resize patch proposal 输入的大小。 2）将 patch 前馈到 conv4，以便为每个filter 生成激活分数。3）将一个类别中 filter 的分数相加，得到类别分数。 4）对于每一聚类，选择生成分数最高的 patch，把这个 patch 当做 part patch。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/4.jpg?raw=true)

图 4 显示了一些狗和鸟类的检测结果。很明显，DomainNet 中的一组 filter 特别注意鸟头，另一组注意身体。同样的，一组 filter 会关注狗头，另一组关注狗腿。  

**构建 part-based 分类器** 将由 part 检测器选择的 patch 输入到 DomainNet 生成激活值。我们拼接不同部分和原始图像的激活值，然后训练 SVM 作为 part-based 分类器。  

该方法包含几个超参数，例如检测滤波器层：conv4，聚类数量：3。实验中，取训练数据的 10％ 作为验证集用 grid search 方法来确认这些超参数。我们发现 conv4 比 conv3 或 conv5 更好，而设置 k > 3 并没有带来更好的准确率。  

### 2.3. The Complete Pipeline  

DomainNet 分类器和 part-based 分类器都是细粒度分类器。然而，他们的功能和优势是不同的，主要是因为他们接受的 patch 不同。使用 selective
search 自下而上生成的是未加工的 patches。从它们中，FilterNet 选择多个视图希望将焦点集中在整个对象上，这些 patches 驱动 DomainNet。另一方面，part-based 分类器选择并专门处理包含可区分的和局部的特征。尽管有些 patches 被两个分类器都使用了，但它们的特征在每个分类器中都有不同的表示形式。最后，我们将 two level attention 模型的预测结果合并，利用以下公式综合两者的优势：  

$final\_score = object\_score + \alpha * part\_score$

其中对象得分是由 object attention 选择的多个 patch softmax 分数的平均值，part 得分是由 SVM 使用拼接特征产生的决策值，$\alpha$ 是通过交叉验证得到的值。实验中，$\alpha = 0.5$。选择具有最高评分的类作为预测结果。  

图 5 显示了合并 two level attention 分类器的完整的 Pipeline。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/5.jpg?raw=true)

## 3. Experiment  

本节介绍性能评估并分析我们提出的方法在三个细粒度分类任务中的表现：  

- ILSVRC2012 中的两个子集，狗数据集（ILSVRC2012 Dog）和鸟类数据集（ILSVRC2012 Bird）的分类。 第一个包含 118 类狗品种的 153,773 幅图像，第二个包含 59 种鸟类的 79,491 张图像。训练/测试分组遵循 ILSVRC2012 的标准协议。两个数据集都被弱注释，只有类标签可用。  

- 广泛使用的细粒度分类基准 Caltech-UCSD Birds 数据集，包含 200 种鸟类的 11,788 幅图像。CUB200-2011 中的每个图像都有详细的标注，包括图像级标注，边界框和关键点标注。  

### 3.1. Implementation Details  

我们的 CNN 架构与流行的 AlexNet 基本相同。有 5 个卷积层和 3 个全连接层。它用于所有实验，除了输出层的神经元数量在需要时被设置为类别的数量。为了公平比较，我们尝试在同一网络体系结构上重现其他方法的结果。当使用 CNN 作为特征提取器时，第一个全连接层的激活作为特征输出。最后，为了证明我们的方法对网络结构是不可知的，并且可以通过网络结构改进分类效果，我们也尝试在特征提取阶段使用更新的 VGGNet。由于时间有限，我们没有复现使用 VGGNet 的所有结果  

ILSVRC 1K 的 Bird 和 Dog 子集用于训练 DomainNet，CUB200 2011 用于微调 DomainNet_Bird。所有用于训练的图像都使用 object-level attention 方法进行增强。  

### 3.2. Results on ILSVRC2012 Dog/Bird  

在此任务中，只有图像级别的类标签可用。因此，需要详细标注的细粒度方法不适用。为简洁起见，我们只会报告狗类的结果，鸟类的结果在性质上是相似的。  

基线是 CNN，但有两种不同的训练策略，包括：  

- CNN_domain：只用来自狗类的图像训练网络。在训练阶段，从整幅图像中随机剪裁 227×227 图像块，以避免过拟合。在测试阶段，将 10 个固定视图（中心，四个角和它们的水平翻转）的 softmax 输出平均值做为最终预测。在这种方法中，没有使用特别的 attention 并且 patches 是 equally 选择的。  

- CNN_1K：网络在 ILSVRC2012 1K 类别的所有图像上进行训练，然后删除不属于狗的 softmax 神经元。其他设置与上述相同。这是一个多任务学习方法，可以同时学习所有模型，包括狗和鸟。这种策略利用更多的数据来训练单个 CNN，并且更好地抵制过拟合，但是在不需要的类别上浪费了容量。  

这些基准数字与我们方法的三种策略进行比较：仅使用对象级别和部分级别的 attention，以及两者的组合。Selective search 提出了数百个 patches，我们让 FilterNet 选择其中的大约 40 个，使用 0.9 的置信度得分。  

表 1 总结了五种策略 top-1 的错误率。事实证明，两条基线的表现基本相同。但是，我们的注意力方法可以实现更低的错误率。与随机裁剪 patch 训练的CNN相比，仅使用对象级 attention 将错误率降低 9.3％。这清楚地表明了对象级 attention 的有效性：现在，DomainNet 专注于从前景对象中学习特定的特征。结合 part 级 attention，错误率下降 28.1％，明显好于基线。单独使用 part 级 attention 的结果不如对象级 attention，因为 part 级 attention 还存在更多模糊性。然而，它实现姿态规范化以抵抗大的姿态变化，这与对象级 attention 是互补的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/6.jpg?raw=true)

### 3.3. Results on CUB200-2011  

对于这项任务，我们首先演示基于对象级 attention 学习深度特征的性能优势。然后，我们针对其他最先进的方法展示完整结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/7.jpg?raw=true)

**学习深度特征的优势** 我们已经证明，受到对象级 attention 训练的鸟类 DomainNet 可以在 ILSVRC2012 Bird 上提供出色的分类性能。假设部分收益来自更好的学习特征是合理的。在这个实验中，我们使用 DomainNet 作为 CUB200- 2011 上的特征提取器来验证这些特征的优势。我们与两个基线特征提取器进行比较，一个是手工制作的核心描述符（KDES），它在使用 CNN 特征之前被广泛用于细粒度分类，另一个是 CNN 特征提取器从 ILSVRC2012 中所有数据进行预训练。我们比较了两种分类 pipelines 下的特征提取器。第一个使用边界框，第二个是在 Zhang 等人提出的（DPD）依靠基于可变形部分的检测器来查找对象及其局部区域。在这两条 pipeline 中，特征送入 SVM 分类器。在这个实验中，CUB200-2011 没有对 CNN 进行微调。如图 6 所示，基于 DomainNet 的特征提取器可以在两条管道 pipeline 上实现最佳结果。这进一步表明，使用对象级别的注意力过滤相关的 patch 是 CNN 学习好的特征的重要条件。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/3/8.jpg?raw=true)

**分类 Pipeline 的优势** 本实验中，使用 CUB200-2011 对 DomainNet 进行了微调，并使用由对象级注意了产生的 patches。表 2 中给出了使用不同标注下的准确率。这些方法分为三组。第一组是我们基于注意力的方法，第二组使用与第一组相同的 DomainNet 特征提取器，但使用不同的 pipeline 和标注，第三组包括最近文献中的最新结果。由于训练数据数量有限，第二和第三组中的大多数比较方法使用 SVM 作为分类器，例如 BBox + DomainNet，DPD，Part RCNN。这些方法的区别在于提取特征的位置。  

我们首先比较使用相同特征提取器的前两组结果，并将性能差异归因于不同的注意力模型。使用原始图像只能达到最低的准确率（58.8％）。这表明了细粒度图像分类中对象和部分注意了的重要性。相比之下，我们的注意力方法取得了显着的改善，两级注意力模型比使用使用人工标注框的方法更好（69.7％比68.4％），并且与 DPD（70.5％）相当。DPD 结果基于我们的特征提取器实现的，它使用可变形的 part-based 检测器，用对象边界框进行训练。标准 DPD pipeline 在测试时也需要边界框来产生相对较好的性能。就我们所知，69.7％ 是弱监管下的最佳结果。  

第三个总结了目前最先进的一些方法。我们的结果比仅使用边界框进行训练和测试的结果要好得多，但与使用局部级标注的方法仍有差距。  

我们的结果可以通过使用更强大的特征提取器来改进。如果我们使用 VGGNet 来提取特征，那么仅仅使用原始图像而没有注意力的基线方法可以提高到72.1％。增加对象级别的注意力，部分级别的注意力以及综合注意力分别将性能提升至 76.9％，76.4％ 和 77.9％。  

## 4. Related Work  

细粒度分类最近被广泛研究，以前的作品主要从三个方面提高识别精度：1. 对象和局部的定位，也可以将其视为对象/局部注意力; 2. 检测到的物体或局部的特征表示; 3. 人为介入。由于我们的目标是自动细粒度分类，因此我们重点关注前两项的相关工作。  

### 4.1. Object/Part Level Attention  

在细粒度分类任务中，可区分性特征主要局限于前景对象，甚至对象的局部，这使得对象和局部注意力成为第一个重要步骤。由于细粒度分类数据集通常使用边界框和关键点的详细标注，因此大多数方法都依赖其中一些标注来实现对象或局部级别的注意力。  

最强有力的监督设置是在训练和测试阶段都使用边界框和关键点标注，这通常用于提升测试性能。为了在细粒度任务上验证 CNN 特征，在训练和测试阶段都假定提供了标注框。使用提供的标注框，提出了几种以无监督或潜在方式学习局部检测器的方法。为了进一步提高性能，局部级别标注还用于训练阶段，以学习强监督的可变形 part-based 模型或直接用于微调预先训练的 CNN。  

我们的工作也与最近提出的基于 CNN 特征的目标检测方法（R-CNN）密切相关。R-CNN 首先通过一些自下而上的注意力模型为每幅图像提出数千个候选区域，然后选择分类得分高的区域作为检测结果。基于 R-CNN，Zhang 等人已经提出了 Part-based R-CNN 利用深度卷积网络进行局部检测。  

### 4.2. Feature Representation  

提高准确率的另一方面是直接引入更具辨别性的特征来表示图像区域。Ren 等人已经提出了 Kernel 描述符，并被广泛用于细粒度分类中。Berg 等人最近的一些作品尝试从数据中学习特征描述。从 ImageNet 数据中预先训练的 CNN 特征提取器在细粒度数据集上的性能也有显著的改善。Zhang 等人通过对细粒度数据集进行微调来进一步改善了 CNN 特征提取器的性能。  

我们的方法采用相同的原则。我们也采用了自下而上的区域提议的策略，就像 R-CNN 和 Part R-CNN 一样。一个区别是我们使用提供多个视图和缩放的相关 patch 丰富了 object-level pipeline。更重要的是，我们整个模型选择最弱的监督，仅依靠 CNN 特征来实现注意力，检测丁武和提取特征。  

## 5. Conclusions  

在本文中，我们提出了一种结合了自下而上和自上而下注意力的细粒度分类 pipeline。对象级别的注意力为网络提供了与输入图像相关的具有不同角度和缩放比例的 patches。这用于细粒度分类的 CNN 特征更好。部分级别的注意力集中在局部可区分特征上，并且也实现了姿态规范化。这两种注意力都可以带来显著的收益，并且它们通过后期融合可以相互补充。我们方法的一个重要优点是，注意力来源于用分类任务训练的 CNN，因此它可以在只提供类别标签的最弱监督环境下进行。这与其他最先进的方法形成鲜明对比，这些方法需要对象边界框或局部标注进行训练或测试。据我们所知，我们在最弱的监督环境下获得了 CUB200-2011 数据集的最佳准确率。  

这些结果是有希望的，同时，指出了一些经验教训和未来方向，总结如下：  

- 处理 part level attention 中的模糊问题。我们目前的方法没有充分利用在 CNN 学到的东西。由于尺寸问题，part 特征可能出现在不同的层中，因此应将不同层的 filter 视为一个整体，以促进强大的 part 检测。  

- 更紧密地整合 object-level 和 part level attention。object-level attention 的一个优点是它可以提供大量的相关 patches 以在一定程度上抵御图像变化。但是，目前的 part level attention 流程并未利用这一点。我们可以借用 multi-patch testing 的思想来实现 part-level attention 方法，从而产生更有效的姿态归一化。  

我们正在研究追求上述方向。

# Look Closer to See Better: Recurrent Attention Convolutional Neural Network for Fine-grained Image Recognition  

## Abstract  

细粒度物体类别识别（比如鸟类）是很困难的，这是因为可区分区域定位（discriminative region localization）和细粒度特征学习（fine-grained feature learning）是很具有挑战性的。现有方法主要都是单独地来解决这些挑战性问题，然而却忽略了区域检测（region detection）和细粒度特征学习之间的相互关联性，而且它们可以互相强化。在本文中，我们提出了一个全新的循环注意力卷积神经网络（RA-CNN），以相互强化的方式对可区分区域注意力（discriminative region attention）和基于区域的特征表征（region-based feature representation）进行递归学习。在每一尺度规模上进行的学习都包含一个分类子网络（classification sub-network）和一个注意力提议子网络（APN）。APN 从完整图像开始，通过把先期预测作为参考，由粗到细迭代地生成区域注意力，同时精调尺度网络（finer scale network）以循环的方式从先前的尺度规格输入一个放大的注意区域（amplified attended region）。所提出的 RA-CNN 通过尺度内分类损失（intra-scale classification loss）和尺度间排序损失（inter-scale ranking loss）进行优化，以相互学习精准的区域注意力和细粒度表征。RA-CNN 并不需要边界框或边界部分的标注，可以进行端到端的训练。我们进行了全面的实验，并显示 RA-CNN 在三项细粒度任务中取得了最佳性能，在 CUB Birds，Stanford Dogs 和 Stanford Cars 的相对准确率分别提高了3.3％，3.7％和3.8％。  

## 1. Introduction  

通过计算机视觉技术识别细粒度类别（例如鸟类，花朵类型，车模型等）已经引起了广泛的关注。由于某些细粒度的类别（例如，eared grebe and horned grebe）只有领域专家才能区分，因此这项任务非常具有挑战性。与普通识别不同，细粒度图像识别需要能够定位和表示下属类别中非常细微的视觉差异，因此可以用于各种各样的应用，例如专家级图像识别，丰富的图像标注等等。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/1.jpg?raw=true)

细粒度识别的挑战主要有两方面：可区分性区域的定位和细粒度特征学习。通过引入 part-based 的识别框架，以前的研究已经取得了令人印象深刻的进展，其通常包括两个步骤：1）通过分析来自神经网络的卷积特征以无监督的方式或通过使用监督的边界框/部分标注来识别可能的目标区域，以及2）提取每个区域的特征并将它们编码成紧凑的向量来进行识别。虽然已经报道了较好的结果，但进一步的改进受到以下限制。首先，human-defined 区域或现有无监督方法学习的区域可能不是机器分类的最佳选择。其次，来自类似的细粒度类别的局部区域存在的细微视觉差异仍然难以学习。我们发现区域检测和细粒度特征学习是相互关联的，因此可以相互补充。如图 1 所示，精确的头部定位可以促进学习可区分的头部特征，进一步帮助确定后脑存在的不同颜色。  

针对以上挑战，我们提出了一种新循环注意力卷积神经网络（RA-CNN），用于无边框/局部标注的细粒度识别。RA-CNN 以相互加强的方式递归地学习可区分区域注意力和基于区域的特征表示。所提出的 RACNN 是一个叠加网络，输入分别是全图像和多尺度的局部细粒度图像。首先，多尺度网络在每个尺度上共享相同的网络架构，但具有不同的参数以适应具有不同分辨率（例如，图 1 中的粗尺度和更精细尺度）的输入。每个级别的学习包括一个分类子网络和一个注意力提议子网络（APN），它可以确保每个尺度都有充分的可区分能力，并为下一个更精细的级别生成一个精确的参考区域。其次，专用于高分辨率区域的精细网络将输入视为放大的区域来提取更细粒度的特征。第三，recurrent network 通过用于分类的尺度内分类损失和 APN 中的尺度间排序损失进行优化。排序损失优化了更精细的网络，以在正确的类别上产生比先前预测更高的置信度分数。  

由于细粒度网络可以循环堆叠，因此 RA-CNN 可以逐渐地从粗到细（例如，从身体到头部，然后到鸟嘴）区分最具区分性的区域。请注意，准确的区域定位可以帮助基于区域的可区分性特征学习，反之亦然。 因此，提出的网络可以从区域定位和特征学习之间的相互加强中受益。为了进一步利用集成学习的优势，通过学习全连接的融合层，将来自多个尺度的特征进行融合以对图像进行分类。就我们所知，这项工作最早开始用多尺度循环注意力网络来进行细粒度图像识别。我们的贡献可以总结如下：  

- 我们通过提出一种新颖的循环注意卷积神经网络架构来解决细粒度识别问题，该架构能够以相互加强的方式同时准确检测可区分区域并有效学习区域特征。
- 我们提出了成对的排序损失来优化注意力提议网络（APN）。与只有标签监督的区域检查器相比，这种设计使网络能够参照以前的尺度逐渐关注更细粒度的区域。
- 我们对三个具有挑战性的数据集（(CUB Birds, Stanford Dogs, Stanford Cars）进行全面实验，并在所有这些数据集上实现优于最先进方法的性能。  

本文其余部分的安排如下。第 2 节回顾了相关工作。第 3 节介绍了所提出的方法。第 4 节提供了评估和分析，其次是第 5 节的结论。  

## 2. Related Work  

关于细粒度图像识别的研究沿着两个维度进行，即可区分性特征学习和复杂的区域定位。  

### 2.1. Discriminative Feature Learning  

学习可区分性特征对细粒度图像识别至关重要。由于深度学习的成功，大多数方法都依赖于强大的卷积深度特征，这些特征在一般和细粒度类别上均比手工特征有明显改善。为了学习更强的特征表示，深度残差网络通过优化残差函数将 CNN 扩展到 152 层，在 ImageNet 测试集中将错误率降低到 3.75％。为了更好地模拟细粒度类别中存在的微妙差异，最近提出了一种双线性结构，通过两个独立 CNN 计算成对特征相互作用来捕获图像局部差异，从而在鸟类分类中实现了最先进的结果。此外，另一种方法提出将 Fisher Vector 的空间加权表示法与 CNN 相结合，该方法在鸟类和狗类数据集中都显示出优异的结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/2.jpg?raw=true)

### 2.2. Sophisticated Part Localization  

以前的工作主要集中在利用边界框和局部的额外标注来定位细粒度识别中的重要区域。然而，手工标注的大量工作使得这项任务对于大规模的实际问题而言并不实用。最近出现了一些针对更普遍情况的方法，并建议使用无监督方法关注重要区域。一种基于视觉注意力的方法提出了一个 two-level 对象和局部的网络，其中局部模板通过聚类方案从 CNN 内部的隐藏表示中学习得到。采用深层滤波器响应和多粒度描述符通过分析来自 CNN 的滤波器响应，以无监督的方式学习一组局部检测器。Spatial transformer 进一步提出了一种动态机制，它可以主动地对图像进行空间转换，以实现更精确的分类。而现有模型对提取小尺寸图像中的局部区域还是有难度的。与我们最相关的作品都建议放大可区分性局部区域以改善细粒度识别的性能。  

## 3. Approach  

在本节中，我们将介绍用于细粒度图像识别的循环注意力卷积神经网络（RA-CNN）。我们将图 2 中的三个尺度网络视为一个例子，并且可以以类似的方式堆叠更精细的尺度。输入循环的从 a1 中的全尺寸图像到 a2 和 a3 中的细粒度可区分区域，其中 a2 和 a3 分别将输入作为来自 a1 和 a2 的参考区域。 首先，将不同尺度的图像输入卷积层（b1 至 b3）以提取基于区域的特征表示。其次，网络通过全连接和 softmax 层（c1到c3）预测分类概率得分以及注意力提议网络（d1，d2）对区域的注意力。所提出的 RA-CNN 通过交替地学习每个尺度的 softmax 分类损失和相邻尺度上的成对的排序损失来优化收敛。  

### 3.1. Attention Proposal Network  

**多任务公式：** 传统的 part-based 细粒度识别框架没有利用经过训练的网络来相互促进定位和识别学习。受近期区域提议网络（RPN）成功的启发，本文提出了一个注意提议网络（APN），其中区域注意力的计算几乎是免费的，并且可以对 APN 进行端对端的训练。  

给定输入图像 X，我们首先通过将图像提供给预先训练的卷积层来提取基于区域的深度特征。提取的深度特征表示为 $W_c * X$，其中 $*$ 表示 convolution, pooling 和 activation 的一系列操作，并且 $W_c$ 表示所有的参数。我们进一步在每个尺度下将网络建模为具有两个输出的多任务公式。 第一项任务旨在生成细粒度类别的概率分布 p，如下所示：  

$p(X) = f(W_c * X)$

其中 $f(\cdot)$ 表示全连接层将卷积特征映射到可以与类别条目匹配的特征向量，并且包括 softmax 层以将特征向量进一步转换为概率。第二项任务是提出为下一个更精细的尺度预测关注区域的一组坐标。通过三个参数将预测的坐标近似为一个矩形区域，表示如下：  

$[t_x, t_y,t_l] = g(W_c * X)$

其中 $t_x, t_y$ 分别表示为正方形的中心坐标，$t_1$ 表示正方形边长的一半。 $g(\cdot)$ 的具体形式可以表示为具有三个输出的两层全连接层，这三个输出是关注区域的参数。请注意，与使用强监督信息(ground truth boxes)的目标检测中的 RPN 网络相比，提出的 APN 的学习以弱监督方式进行训练，因为 part-level 级别的标注通常很难获得。具体的学习过程和损失函数将在 3.2 节中介绍。  

**关注区域的定位和放大：** 一旦有了关注区域的定位，我们裁剪并放大关注区域，以更高的分辨率更精细地提取更多的细粒度特征。为了确保 APN 可以在训练中进行优化，我们通过提出二维 boxcar 函数的变体作为注意例 mask 来近似裁剪操作。由于连续函数的性质，mask 可以选择前向传播中最重要的区域，并且易于在向后传播中进行优化。  

假设原始图像的左上角是像素坐标系的原点，其坐标系的 x 轴和 y 轴分别从左到右和从上到下定义。相应的，我们可以以一下方式从关注区域的左上角($t_l$)和右下角($t_r$)进行参数化：  

$t_{x(tl)} = t_x - t_l, \ t_{y(tl)} = t_y - t_l$
$t_{x(br)} = t_x + t_l, \ t_{y(br)} = t_y + t_l$

基于上述表示，裁剪操作可以通过在较粗尺度的原始图像和关注 mask 之间的 element-wise 的乘法来实现，其可以被计算为：  

$X^{att} = X \bigodot M(t_x,t_y,t_l)$

其中 $\bigodot$ 表示 element-wise 乘法，$X^{att}$ 表示裁剪出的关注区域，$M(\cdot)$ 用作注意掩码，具体形式为：  

$M(\cdot) = [h(x - t_{x(tl)}) - h(x - t_{x(br)})] \cdot [h(y - t_{y(tl)}) - h(y - t_{y(br)})]$

$h(\cdot)$ 是一个具有参数 k 的 logistic function：

$h(x) = 1/\{1+{exp}^{-kx}\}$

理论上，当 k 足够大时，对数函数可以看作是一个阶梯函数，并且在 x 和 y 维的整条实线上，二维 boxcar function($M(\cdot)$) 为零，除了单个区域 (x 从 $t_{x(tl)}$ 到 $t_{x(br)}$, y 从 $t_{y(tl)}$ 到 $t_{y(br)}$)中它等于 1。 使用 boxcar function 有两个好处。 首先，boxcar 函数可以很好地逼近剪切操作，以选择从较粗尺度网络预测的最重要区域。其次，boxcar 函数在关注区域和框坐标 $\{t_x，t_y，t_l\}$ 之间建立解析表示，这在后向传播中优化框参数时是必要的。  

尽管已经定位到了关注区域，但有时候仍然难以从高度局部区域提取有效的特征表示。因此，我们通过自适应缩放将区域进一步放大到更大的尺寸。具体来说，我们使用双线性插值法将 $X^{att}$ 中最近四个输入映射为放大的输出 $X^{amp}$，其由下式给出：  

$X_{(i,j)}^{amp} = \sum_{\alpha,\beta = 0}^1 \mid 1-\alpha-\{i/\lambda\} \mid \mid 1-\beta-\{j/\lambda\} \mid X_{(m,n)}^{att}$

其中 $m = [i/\lambda] + \alpha, n = [j/\lambda] + \beta$，$\lambda$ 是上采样因子。  

### 3.2. Classification and Ranking  

所提出的循环注意 CNN 通过两种类型的监督来进行优化，即尺度内的分类损失和尺度间成对的排序损失，交替的生成精确的区域关注和学习更细粒度的特征。具体而言，我们将多任务损失后的目标函数最小化。图像样本的损失函数定义如下：  

$L(X) = \sum_{s=1}^3 \{L_{cls}(Y^(s),Y^*)\} + \sum_{s=1}^2 \{L_{rank}(p_t^{(s)},p_t^{(s+1)})\}$

其中 s 表示每个尺度，$Y^(s)$ 和 $Y^*$ 分别表示在特定尺度上预测的类别和真是标注的类别。$L_{cls}$ 代表分类损失，它主要优化图 2 中卷积和分类层的参数，以确保在每个尺度上具有充分的区分能力。训练通过 softmax 函数在整体训练样本上拟合分类标签来实现。此外，成对排序损失 $L_{rank}$ 中的 $p_t^{(s)}$ 表示正确类别标签 t 上的预测概率。具体来说，排序损失由下式给出：  

$L_{rank}(p_t^{(s)},p_t^{(s+1)}) = max(0, p_t^{(s)} - p_t^{(s+1)} + margin)$

这在训练中强制 $p_t^{(s + 1)} > p_t^{(s)} + margin$。这样的设计可以使网络能够将粗尺度的预测作为参考，并通过执行更精细的网络来逐渐接近最具有区别性的区域，以产生更有信心的预测结果。  

### 3.3. Multi-scale Joint Representation  

一旦提出的 RA-CNN 在每个尺度上被训练，我们就可以从全尺寸图像获得多尺度表示。特别地，图像 X 可以由一组多尺度描述符来表示：{F1，F2，... FN}。其中，Fi 表示从分类网络全连接层生成的特定尺度的特征描述符，N 是尺度总数。为了充分利用特征集成的优势，我们首先将每个描述符独立地归一化，并将它们连接在一起形成具有 softmax 函数的全连接融合层，用于最终分类。Softmax 函数代替支持向量机的应用主要是为了特征提取和分类的技术一致性，使得我们可以将多尺度描述符和端到端的分类整合到测试中。此外，我们已经证实 softmax 和线性支持向量机可以产生可比较的分类结果。  

### 3.4. Implementation Details  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/3.jpg?raw=true)

**训练策略：** 为了更好地优化注意力定位和细粒度分类，我们采取以下交替训练策略。  

步骤1：我们用相同的经过 ImageNet 预训练的 VGG 网络初始化图 2 中的卷积/分类层（b1 至 b3 和 c1 至 c3）。  

步骤2：我们考虑原始图像边长的一半的正方形($t_x，t_y，t_l$)。通过搜索原始图像中的区域来选择正方形，在最后的卷积层中具有最高的响应值。我们可以通过类似的方式在第二个尺度上分析卷积响应来获得更小的正方形。这些选择到的正方形用来预训练 APN 网络来得到图 2 中的参数(d1,d2),用来学习将卷积特征转换为{tx,ty,tl}。  

步骤3：我们交替的优化上述两个步骤中的参数。具体而言，我们保持 APN 参数不变，并优化三个尺度下的 softmax 损失以进行收敛。然后，我们固定卷积/分类层中的参数，然后优化两个 APN 中的排序损失。这两部分的学习过程是迭代的，直到两种类型的损失不再发生变化。此外，每个尺度的 tl 被限制为不小于前一个 tl 的尺度的三分之一，以避免当 tl 太小时物体结构的不完整性。  

**优化注意力学习：** 我们通过计算 tx，ty，tl 上的导数，并显示对区域剪裁的影响来说明注意力学习的机制。由于所提出的排序损失对 tx，ty，tl 的导数具有相似的形式，因此我们以 tx 为例，并通过向后传播中的链式规则计算导数，它由下式给出：  

$\frac {\partial L_{rank}}{\partial t_x} \propto D_{top} \bigodot \frac {\partial M(t_x,t_y,t_l)}{\partial t_x}$

其中 $\bigodot$ 表示 element-wise 乘法，$D_{top}$ 表示以特定比例反向传播到输入层的导数，表示每个像素相对于整个网络激活的重要性。  

## 4. Experiments  

### 4.1. Datasets and Baselines  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/4.jpg?raw=true)

**数据集：** 我们在三个具有挑战性的细粒度图像识别数据集上进行实验，包括 CaltechUCSD Birds（CUB-200-2011），Stanford Dogs 和 Stanford Cars。表 1 总结了具有分类和数据拆分的详细统计数据。  

**基线：** 根据是否使用人工定义的边界框（bbox）或局部标注，将比较的方法分为两类。下面，前五种方法使用人工监督，后八种方法基于无监督部分学习方法。与他们比较，是由于他们在这两个类别的分类上得到了最好的结果。所有的 baselines 如下：  

- DeepLAC：深度定位，对齐和分类，使用姿势对齐的局部图像进行分类。
- SPDA-CNN：语义部分检测和 abstraction proposes 通过检测/分类网络生成区域候选并提取特征。
- Part-RCNN：通过 part annotations 扩展 R-CNN 框架。
- PA-CNN：pose normalized 的方法，通过估计目标姿态计算局部特征。
- PN-CNN：姿态归一化CNN提出通过估计物体的姿态来计算局部特征。
- PDFR：挑选具有区分性的 feature map，学习局部检测器。
- MG-CNN：多尺度描述符，学习多个感兴趣区域的特征。
- ST-CNN：空间变换器网络。
- TLAN：两级关注网络，提出对象和局部域分类。
- DVAN：不同的注意力网络。
- FCAN：全卷积注意力网络通过强化学习自适应地选择多个任务驱动的视觉注意力。
- B-CNN：双线性-CNN提出捕获用于分类的成对特征的交互。
- NAC：一系列神经激活，通过计算神经激活模式来查找局部区域。  

由于粗尺度中的对象尺寸较小，输入图像和提议关注图像在训练中分别调整为 448×448 和 224×224 像素。我们对鸟类和汽车数据集使用 VGG-19（在ImageNet 上预先训练），对狗使用 VGG-16 作为与基线相同的设置。该模型已经在 https://github.com/Jianlong-Fu/Recurrent-Attention-CNN 公开发布。  

### 4.2. Experiments on CUB-200-2011  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/5.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/6.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/7.jpg?raw=true)

**Attention localization：** 为了定性分析我们展示了多个尺度下通过 APN 网络计算出来的关注区域。在图 4 中可以观察到，这些第二和第三尺度上的计算出来的关注区域可以区分相应的类别，并且比第一尺度更容易分类。结果与人类感知一致，认为离得更近对细粒度物体的识别是有帮助的。

由于 APN 网络是自动学习最具有可区分性的区域，而不是从标注的边界框中回归得到的区域，我们在分类精度方面对注意力定位进行了定量比较。为了公平比较，所有的方法都使用 VGG-19 模型，但用了不同的定位算法。我们用第二尺度的网络来生成结果，因为在这个尺度上的关注区域可以保存鸟的整体结构及局部特征，如图 4 所示。首先，我们可以得到与表 3 中列出的需要人为标注边界框方法具有可比性的结果，PA_CNN 和 MG_CNN 准确率分别为 82.8% 和 83.0%，我们的 PA_CNN 准确率为 82.4%。其次，与现有的无监督局部学习方法相比，我们可以取得更好的结果。FCAN 和 MG-CNN 是两个与我们相关的工作，也使用多尺度/粒度的特征组合方案。与 FCAN 和 MG_CNN 相比，我们分别提高了 8.3% 和 3.6%。此外，第三行展示了用了关注区域但没有使用排序损失时 RA_CNN 的效果，我们可以知道排序损失在优化区域注意力时的关键作用。

**细粒度图像识别：** 跟据是否使用人工定义的边界框（bbox）/局部标注来比较两种 baseline。PN-CNN 使用了强大的有监督训练(人工定义的边界框和局部标注)。 B-CNN 使用边界框具有非常高维的特征表示（25万维）。如表 3 所示，即使没有 bbox 和局部标注，所提出的 RA-CNN（比例 1 + 2 + 3）也可以与 PN-CNN 和 B-CNN 获得可比较的结果，这证明了方法的有效性。与无监督方法 PDFR 相比，没有额外的 Fisher 向量学习，通过我们的完整模型 RA-CNN（尺度1 + 2 + 3）可以获得 3.3％ 的相对准确度增益。我们甚至超过 B-CNN(w/o anno.) 和 ST-CNN，它们使用高维特征或更强的初始网络作为基准模型，几乎有 1.5％ 的相对准确性增益。虽然 FCAN(w/o anno.) 和 DVAN 提出了类似的想法来放大分类区域，但我们的框架可以改善 4.1％ 和 8.0％ 的准确率。请注意，RA-CNN（scale 2）比 scale 1 级的 VGG-19 结果优越，相对增益5.9％，对细粒度物体识别“looking closer”是有必要的。  

### 4.3. Experiments on Stanford Dogs  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/8.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/9.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/4/10.jpg?raw=true)

表 4 总结了Stanford Dogs 数据集的分类准确率。第一个尺度的 VGG-16 将原始图像作为输入，并且获得了 76.7％ 的识别准确度。依靠准确的注意力定位，RA-CNN（第2级）的识别准确率达到 85.9％，相对增益为 12.0％，显著改善。通过结合两个尺度和三个尺度的特征，我们可以将性能分别提高到86.7％ 和 87.3％。与 DVAN 和 FCAN 两种最相关的方法相比，相对准确率分别提高 7.1％ 和 3.7％。这种改善主要来自准确的注意力定位，这在图 5 中得到了证明。该图证明，注意到的区域大部分位于狗头上，这与以前的研究一致。  

### 4.4. Experiments on Stanford Cars  

表 5 总结了 Stanford Cars 的分类准确率。由于细微的差异，例如不同的前后设计，不同的汽车模型难以被识别。尽管 VGG-19 的 scale 1 只能达到84.9％ 的准确度，但在将区分性区域注意力放大到更精细的比例之后，性能可以提高到 90.0％。我们利用特征集合的功能获得 92.5％ 的最高识别精度，集成了原始图像，放大的整车和前后区域的特征。我们可以从图 5 中分析出，提议的关注建议网络能够对代表性参与区域进行定位，例如奥迪和道奇独特的前部设计，以及 Smart 可爱的背部设计。与最先进的方法相比，我们的全模型 RA-CNN（scale 1 + 2 + 3）上超过了 DVAN 和 FCAN (w/o anno.)。我们也获得比高维 B-CNN 更好的结果，甚至可以实现与 PA-CNN 相媲美的性能，它使用了人类定义的边界框。  

## 5. Conclusion  

在本文中，我们提出了一种用于细粒度识别的循环注意卷积神经网络，它递归学习多尺度下的区分性区域定位和基于区域的特征表示。所提出的网络不需要用于训练的边界框/局部标注，并且可以被端对端训练。大量的实验证明了在鸟类，狗和汽车上的注意定位和细粒度识别任务方面的优越性能。未来，我们将进行两个方向的研究。首先，如何同时保存全局图像结构和模型局部视觉特征，以不断提高更精细的性能。其次，如何将多个区域的注意力集成到更复杂的细粒度类别中。

# Fine-grained Recognition in the Wild: A Multi-Task Domain Adaptation Approach  


## Abstract  

虽然细粒度的目标识别是计算机视觉中的一个重要问题，但目前的模型不可能准确地对野外物体进行分类。这些完全监督的模型需要额外的注释图像来对每个新场景中的对象进行分类，这是一项不可行的任务。但是，电子商务网站和实地指南等资源为许多课程提供注释图像。在这项工作中，我们研究细粒度域适应，作为克服容易获得的注释图像和现实世界之间的数据集转换的一个步骤。在细节设置中，尚未研究适应性，可以使用注释（如属性）来提高性能。我们的工作使用基于属性的多任务自适应损失来将准确度从4.1％的基线提高到半监督适应案例的19.1％。先前的领域适应工作已经在诸如[46]的小数据集上进行基准测试，对于一些领域总共795个图像或者诸如由数字组成的[41]的简单数据集。我们对一组新的具有挑战性的细粒度数据集进行了实验，这些数据集包括从电子商务网站和Google街景视图中抽取的2 657个汽车类别的1,095,021图像  

## 1. Introduction  

图像识别的最终目标是识别世界上的所有物体，因为它们出现在自然环境中。 一项更为艰巨的任务是细粒度识别，旨在区分同一类别中的物体（例如不同的鸟类或汽车品牌）。 目前的最先进的细粒度分类方法[2,5,8,33]关注完全监督学习机制：一种设置，其中人类注释图像可用于所有感兴趣的对象类别。 为了实现这些方法，已经提出了数据集来训练识别所有类别和场景的模型[15,35,54]，或者专注于细粒度识别任务[51,30,52,42,34]。  

对这些数据集进行训练的模型在评估诸如[15,44]等基准任务时能够胜过人类。 但是，这种评估模式忽略了现实世界对象分类模型发展的关键挑战。 也就是说，像ImageNet或Birds这样的固定数据集提供了一个稀疏而有偏见的世界样本[48]。 因此，为了在现实环境中实现可比较的性能，用这些数据集训练的完全监督模型需要来自每个新场景的附加注释数据。 但是，在不断变化的真实世界环境中采集拍摄所有可能出现的物体的图像是不可行的。 大量可能的图像使得为现实世界中的每个对象类别获得标记示例成本高得惊人。 而且，当我们考虑对细粒度类别进行识别时，这种标注负担会被放大。 在这种情况下，只有专家才能为我们的算法提供标记数据。  

幸运的是，我们可能想要识别的许多对象都有免费可用的配对图像和类别标签。 例如，现场指南中的图像和注释可用于训练识别野外各种鸟类的模型（图1（顶行））。 类似地，电子商务网站上的带注释的汽车图像可用于训练区分非结构化城市环境中不同类型汽车的模型（图1（中间行））。 然而，这些来源的图像与我们在现实世界中可能遇到的图像有不同的统计数据。 而这种统计差异会导致模型性能的显着降低[48,46,4]。  

在这项工作中，我们研究细粒度域适应，作为克服容易获得的注释图像和现实世界之间的数据集转换的一个步骤。 据我们所知，适应尚未在精细的环境中研究，其中获得图像标注特别昂贵。 在这种情况下，我们的许多类别可能会以某种已知的分层方式相互关联。 例如，多个不同的汽车品种可以共享相同的车身类型或相同的品牌。  

我们的贡献有两个方面：首先，我们提出了一种新的多任务适应方法，明确受益于这些已知的跨类别关系。 我们的模型包含一个多任务适应目标，它同时学习和适应属性和类别级别的识别。 我们首先表明，我们的目标有效地规范了源训练，从而改善了源模型对目标领域的推广。 然后，对于半监督适应的任务（即，只有目标域中的类别的子集可以使用类别标签时），我们利用标签通常针对所有属性存在的事实。 例如，虽然1998年本田雅阁轿车的注释目标图像可能无法使用，但其他本田和轿车的图像可能在我们的数据集中。 通过这种方式，我们能够在类和属性级应用不同的自适应技术。  

我们的第二个贡献表征了大规模细粒度汽车数据集的领域适应。 虽然这个数据集是由[27]在细粒度检测中引入的，但它并未用于适应。 我们使用我们的方法在2,657个类别（总共71,030个图像）中的170个子集上执行实验，并显示出显着改进的性能。 尽管视域适应已经得到很好的研究[46,3,28,49,24]，但大多数方法都是针对由几十个对象类别和数百个图像构成的相对较小的数据源进行调整[46，20,41]。 使用这样的小数据集来开发自适应算法使得难以可靠地对这些算法进行基准测试。 据我们所知，我们的工作是第一个在大规模，真实世界的数据集和细粒度场景中研究这个重要问题的。  

## 2. Related Work  

细粒度的物体识别。 虽然细粒度图像识别是一个研究得很好的问题[2,5,8,10,11,9,16,17,19,26]，但其真实世界的适用性受到有限的可用数据的阻碍。 像[33]这样的工程已经使用大规模噪声数据来训练最先进的细粒度识别模型。 但是，这些模型不太可能推广到现实世界的照片，因为它们通过从现场指南或产品镜头获得的图像进行训练。 同样，标准细粒度数据集如[51]和[6]是从单个域派生的。 由于现实世界与这些数据集之间物体外观的巨大差异，对这些图像进行训练的模型不可能很好地推广到真实世界的物体。  

域适应。域适应工作可以增强在一个域上训练的模型（如产品镜头图像）的性能，并将其应用于不同的域（如真实世界的照片）。自[4]提供的理论框架以来，许多计算机视觉工作已经公布了无监督域自适应算法：即在训练期间没有标记目标图像可用的任务[53,39,23,1,7,50]。大多数方法都努力学习一个具有域不变特征的分类器[49,25,38]。 Long等人放松源图像和目标图像的单个分类器的假设，而是使用具有残余连接的2个分类器[39]。虽然这些作品专注于无监督领域适应，[49]执行半监督适应，从带有标记目标图像的类到没有。据我们所知，在细粒度环境中没有关于视觉适应性的研究。我们的工作建立在[49]的方法上，表明在这种情况下属性级软标签转移和领域混淆显着提升了性能。  

属性，结构化数据和多任务学习。 在[47]中已经使用属性来改善对象分类，并在[43,37]中执行零点学习。 科迪罗夫等人。 当图像源自多个域时，[31]使用稀疏编码和子空间对齐技术来执行零镜头学习。 我们从这些作品中汲取灵感，并利用属性来提高无监督和半监督领域适应的性能。 与[32]对用户指定属性的适应形成对比，我们使用不同细粒度类别之间共享的标签来促进课程级别转移。 虽然以前的作品如[45,21,18]关注属性学习，但我们的目标是通过使用地面属性属性标签来改善适应性。  

我们强化属性和类别预测之间一致性的方法在精神上与一些利用标签结构的作品相似[14,12]。 [14]使用层次和排除（HEX）图来封装标签对之间的语义关系。 我们使用预测的标签分布而不是硬约束之间的KL散度损失。  

最后，一些先前的研究表明，学习多个任务可以提高每个任务的泛化能力。 例如，[13]发现用于分割的多任务网络将对象检测结果作为双产品提高。 同样，[22]表明，学习翻译多种语言的机器在每种语言上表现更好。 我们观察到类似的结果，其中使用属性的多任务适应方法提高了课程级别的表现。  

## 3. Multi-Task Domain Adaptation for FineGrained Recognition  

在细粒度分类设置中，为每个类别获取标签是不可行的。 但是，课程通常会共享属性。 例如，比格犬和杰克罗素梗犬都是小狗，而大胡子牧羊犬和阿富汗猎犬都是毛茸茸的狗。 在通用对象分类设置中，可以使用分类树（如WordNet）对分类进行分组并获取分层结构中多个级别的标签。 因此，虽然目标域可能没有每个叶节点类的标签，但我们更可能在层次结构中的较高级别注释了图像。  

我们将这些附加注释用于多任务目标，提供正规化和额外的监督。 具体而言，我们最小化由细粒度和属性级别的softmax分类损失组成的多任务目标。 在图2所示的我们的体系结构中，除了类别级别分类之外，这还通过具有执行属性级别的多个独立的softmax层来实现。 我们添加一个属性一致性损失来防止独立分类器预测相关标签。 图2中任何无监督的自适应损失（表示为UA）都可以与我们的方法结合使用。 同样，当目标标签可用于某些类别时，任何半监督自适应丢失（表示为SSA）可以添加到类别和属性级别。 在这里，我们将我们的方法应用于[49]以评估其功效。  

### 3.1. CNN Architecture for Multi-Task Domain Transfer  

我们给出了我们的半监督域自适应架构的概述，如图2所示。我们的模型使用所有类的注释源图像进行训练，我们将其表示为{xS，yS}，并且标记和未标记的目标图像{xT，yT}。 xS，xT分别是源图像和目标图像样本，yS，yT是它们的关联标签。 我们的目标是训练一个模型，将图像{xT}分类为没有标记目标图像的细粒度类别。 我们将目标图像的数量表示为NT，并将标记的目标图像的数量表示为NT L。NT L = 0和NT L = NT分别在无监督和完全监督的自适应设置中。 只有目标图像的一个子集在半监督适应设置中被标记，导致NT L \<NT。  

除了类标签yS，yT之外，我们还分别对源图像和目标图像分别具有属性级别注释ySa，yT a。 对于每个属性a，至少有与每个类c一样可用的标记源和目标图像。 这意味着即使没有标记的目标图像可用于类c，也存在与c具有相似属性的类的标签。 我们使用3个组件来优化多任务丢失：在细粒度和属性级别上的softmax分类损失，属性一致性损失以及任何无监督或半监督的适应性损失。  

### 3.2. Classification Loss  

我们从CNN开始，遵循[36]的架构，以{xS，yS}和{xT，yT}作为输入。 我们将这个分类器的参数表示为θrep。 让每个属性都有一个K类别。 我们有Na属性分类器fan由θan参数化，n = 1 ... Na。 这些分类器对我们的CNN产生的图像特征图f（x，y;θrep）进行操作。 Na是属性的数量，x，y分别是输入图像及其相关标签。 我们最小化Na softmax损失：  

其中ya是图像x和属性a的基础真实标签，pa = [pa1，... paK]是属性分类器扇的激活的softmax。 即，p = softmax（θT和f（x;θrep））。  

除了属性级别softmax损失外，我们还将细粒度级别的softmax分类损失降至最低。 对于K类，以及通过在特征映射f（x，y;θrep）上操作的θC参数化的细粒度分类器，我们使损失最小化：  

我们最终的多任务softmax损失是属性和细粒度softmax损失的加权和。 为简化符号省略参数，  

### 3.3. Attribute Consistency Loss  

尽管我们的属性和类别分类器是使用地面实况标签独立训练的，但我们的管线迄今对这些分类如何相互关联没有限制。也就是说，细粒度分类器可以输出属性与属性分类器预测的属性不同的类。但是，我们知道细粒度类的属性应该与独立属性分类器预测的属性相同。为了实施这种结构，我们添加了一个属性一致性损失，该损失惩罚了由细粒度属性分类器和属性分类器预测的属性之间的差异。我们将由属性分类器an预测的属性分布和由细粒度分类器推断出的属性分布之间的KL散度的对称版本最小化。我们的过程在图3中可视化。对于每个属性a，我们首先将类间的分数（[36]中的输出fc8）转换为该属性的类别间的分数。令f = [f1，... fk]由k个类的分数组成。为了获得跨属性类别的分数，我们对属于来自相同属性的类的f值进行平均。平均而不是简单地加入，分数减轻了数据集偏差的影响，其中一些类和属性更频繁地出现。然后，我们使用计算出的属性分数计算属性a，pa = [pa1，...，paK]的属性类别的softmax分布。  

我们将每个属性a的一致性损失定义为pa和pa之间KL散度的对称版本：  

其中属性a具有3.2中定义的K类别。 由于我们不是试图匹配参考分布，而只是最小化两个分布之间的距离，所以我们在损失中使用对称版本的KL散度而不是交叉熵损失。 为了简化省略参数，最终一致性损失Lconsistency是每个属性损失的加权和：  

### 3.4. Augmenting Existing Adaptation Algorithms with Attribute Loss  

我们可以用我们基于属性的损失来扩充任何现有的自适应算法，以在属性和类级别上执行自适应。 在这里，我们描述了我们如何将我们的方法应用于[49]。 为了在[49]中使用我们的方法，我们添加了[49]中介绍的领域混淆和软标签损失。 软标签丢失仅用于半监督设置，其中有标签的目标图像可用于某些类别。 然而，除了在细粒度级别上的软标签损失Lcsof t之外，我们还将每个属性a的软标签目标Lasof最小化。 这使我们能够利用存在的类别的属性级别注释而没有标记的目标图像。 将领域混淆损失表示为Lconf，我们的最终目标是Lcsof t，Lasof t，Lconf，Lsof tmax和Lconsistency的加权和。  

## 4. Evaluation  

我们在两个数据集（最近提出的大型汽车数据集[27]和办公数据集[46]）上评估我们的多任务适应算法，其中增加了来自WordNet [40]层次结构的属性。 为了测试我们的属性等级适应方法的有效性，我们通过添加属性等级损失来修改现有的域适配方法DC [49]。  

我们在所有的实验中都使用Caffe [29]。 我们的仅源模型使用释放的CaffeNet模型用ImageNet权重初始化[29]。 对于汽车数据集的实验，在计算软标签损耗时，我们在所有损耗和温度为2时使用相同的权重。 我们将所有实验的学习率设置为0.0001，并将发布我们的自定义图层以优化KL散度损失。 对于办公室数据集的实验，除了权重设置为0.1的域混淆丢失外，我们将所有损失权重设置为1  

### 4.1. Large scale car dataset  

在[27]中介绍的汽车数据集包括来自4个来源的crailslist.com，cars.com，edmunds.com和Google街景视图中2,665种汽车的1,095,021幅图像。 我们将来自craigslist.com，cars.com和edmunds.com的图片称为网页图片，将来自Google Street View的图片称为GSV图片。 如图6所示，网页图像中的汽车较大，且通常未被遮挡，而GSV中的汽车较小，模糊且遮挡。 图4显示了图像大小的差异，图4显示了GSV和网页图像中边界框大小的直方图。 姿态，视点，遮挡和分辨率的这些大的变化使得该数据集适合于域适应的研究，特别是在细粒度设置中。 除了分类标签之外，每个班级都伴随着元数据，例如车型，型号车身类型和汽车的制造国。  

### 4.2. Quantifying Domain Shift on the Car Dataset  

在任何适应性实验中，首先了解不同数据来源之间差异的性质至关重要。 按照[46]设定的标准，我们通过训练一系列模型并评估内部和跨域，来量化汽车数据集中的这种变化。 我们对包含170个数据集中最常见类的子集执行所有实验，特别是那些每个类至少有100个目标图像的子集。 这确保我们有足够的图像来可靠地评估我们的算法。  

特别是，我们训练纯源模型，发现在源网域（73.9％）内进行评估时，准确性相对较高，但在GSV目标域内评估时性能会出现灾难性的下降。为了帮助分析我们的半监督域适应实验，我们使用所有可用的GSV标签来训练仅有目标的模型。这个模型作为我们的适应实验的一个预言，它使用一组缩减的标记图像。如Tab。所示。 1，仅目标模型显着优于我们的仅源模型，表明两个域之间有大的转变。最后，我们训练联合完全监督的源和目标模型，以测试web和GSV数据是否互补。事实上，即使是完全监督的目标唯一模型，联合模型也要胜过它。这表明来自源域的带注释的图像将是用于训练对目标图像进行分类的模型的有用资源。因此，在下一组实验中，我们评估了我们的适应解决方案。  

对于这些实验中的每一个，我们都使用细粒度类以及制作模型和体型标签来训练模型。 在用于我们实验的数据集的子集中有10种体型，17种制造和89种模型。  

### 4.3. Multi-Task Adaptation on the Car Dataset  

真实世界领域适应性流水线应该利用标记目标图像的可用性来提高流行的细粒度对象的性能，以改善难以获得标签的类别的分类性能。 有了这个动机，我们将目标数据划分为标记和未标记集合，以执行半监督域适应实验。 我们首先根据他们拥有的目标图像的数量对细粒度类进行排序。 然后，我们将具有最高数量标签的目标类别（85个类别）的前50％的图像与所有类别的源图像一起用作标记的训练数据。 我们的测试数据包含50％的标签数量最少的图像。 因此，在保持半监督的适应性实验中使用的模型没有使用来自保持的类别的标记目标图像。  

表2显示了各种基线方法以及我们的架构的分类精度。 我们的基准线是仅源和DC [49]自适应模型。 我们还将我们的完整模型与没有属性一致性损失的模型进行比较 在所有情况下，我们的属性级别适应机制都会大幅提升性能。 例如，在无监督的适应情景中，我们看到〜10％的增益。 为了确保我们的属性损失确实有助于适应，并且不单单改善基线分类器，我们还培训了一个CNN，它只包含了我们损失的非适应成分：即Lsof tmax和Lconsistency。 虽然属性确实改善了基线模型（准确度从9.28％跳到14.37％），但它们也改善了适应性。 例如，领域混淆可将准确度提高5％而无需属性，但此属性可提升至10％左右。  

我们在半监督的适应环境中看到了与我们的方法类似的收益。 除了网页图像之外，使用GSV类别的标注子集进行的训练通常会降低所支持的GSV类别的性能; 该模型过度适应标记的GSV类，变得不那么普遍。 虽然领域混淆和软标签损失可以解决这个问题，但是当这些方法与属性级别转移结合使用时，我们看到了最显着的改进：准确度从12.34％提高到了19.11％。 这证实了我们的直觉，即使用属性标签可以帮助我们的分类器学习领域不变特征。  

### 4.4. Multi-Task Adaptation on the Office Dataset  

尽管我们的属性级适应方法最适合细粒度设置，但我们还测试了其在办公室数据集上的效果[46]，因为没有其他细粒度的适应性数据集。 办公室数据集包括在办公室周围找到的31类物品（如背包，电脑，台灯和剪刀）。 对于这些对象中的每一个，都可以从3个域获得图像：Amazon，WebCam和DSLR。 尽管2010年推出的这个数据集仍然是今天使用的标准适应基准，但它的尺寸远小于我们实验中使用的汽车数据集。 例如，WebCam域总共包含785个图像（跨越31个类）。  

由于办公室数据集没有属性级别注释，因此我们使用具有不同粒度的类标签来评估我们的多任务适应方法。 我们用WordNet层次结构[40]中父级，祖父级和祖父级节点的类名注释每个图像。 因此，除了其类别标签外，每个图像还有3个标签，分别由3,7和19个类别组成。 我们使用这些附加标签代替我们的多任务适应方法中的属性。 我们的源域是亚马逊，目标是WebCam  

标签。 4和Tab。 5分别显示了我们对无监督和半监督情景的结果。 使用我们的多任务适应方法增加两个基线可以提高无监督和半监督设置的性能。 这表明我们的多任务方法不仅仅局限于属性，并且可以在任何带有标签层次结构的场景中使用。  

n任何具有标签层次结构的场景。 尽管如此，我们的方法在办公室数据集上的性能增益远远低于汽车。 虽然汽车属性在视觉上具有信息性，但WordNet标签可能不是。 例如，自行车和背包都共享节点“容器”，尽管它们的视觉外观非常不同。 我们未来的工作计划探索获得视觉上独特属性标签的其他方法  

### 4.5. Analysis  

我们的模型导致大多数细粒度类别的性能显着提高。 如图7所示，75％的保留类别在[49]中的准确性增加。 同样，在无监督设置下，我们的模型可以提高66％目标类的性能。 14％和16％的班级的准确性没有变化，而10％和18％的班级分别看到两种制度的表现下降。  

而如图5所示，每个细粒度类最多有500个标记目标图像， 图5（B），（C）和（D）显示某些属性具有多达12,000个标记的图像。 虽然我们没有使用任何具有细粒度标签的目标图像作为训练数据，但是在训练数据中存在具有共享属性的类作为测试数据。 因此，我们希望我们的方法能够提高具有多个属性标签的类的准确性。 图8顶部显示了前三类的示例图像，在汽车数据集的半监督设置中具有准确性增益。 这些课程有体型小型货车，扩展驾驶室和SUV：排名前4位的身体类型中有3位具有最高数量的标记目标训练图像  

相反，使用我们的方法导致精确度损失最高的班级是一个乘务员出租车：在我们的训练集中，只有57个标有GSV的乘员出租车图像。 令人惊讶的是，在精度损失最严重的三类中，有两类是轿车。 尽管轿车在我们的训练集中标记的GSV图像数量最多（因此预期会看到准确性增加），但这两个类别中的一个具有243个源训练图像。 图10绘制了相对准确度增益（与[49]相比）与每班有标记的源训练示例的数量。 我们的方法可以提高标注训练数据的班级的准确性。 我们测量每个类别的标签数量与准确性增益之间的相关性为-0.29。  

最后，图9显示了GSV测试集中的示例图像及其在无监督设置下的训练集中的对应最近邻居。 对于每个示例图像，我们使用[49]训练的基线模型和我们的多任务方法来计算其特征激活。 我们检索fc7激活最小化|| L2 ||的训练集中的图像 到示例图像的fc7激活的距离。 虽然我们的基于属性的分类器在与目标图像相同的类中检索图像，但基线适应模型返回错误类中的最近邻居。  

## 5. Conclusion  

我们提出了一个用于半监督域自适应的多任务CNN体系结构。 我们的管道利用细粒度类共享属性的事实，这些属性可以帮助将知识从培训中看到的类转移到那些不知道的类。 我们在由大约1M图像和2,657个汽车类别组成的大规模细粒度数据集的子集上评估了我们的方法。 来自多个域的大量标记图像使得该数据集适合于适应性研究。 我们还使用WordNet的附加标签对标准数据集进行了评估。 今后，我们计划改进我们的方法，以便在适应中引入属性，并在属性标签不可用的设置中执行分层适应。

https://arxiv.org/pdf/1407.3867.pdf  

https://arxiv.org/pdf/1605.06878.pdf

https://www.microsoft.com/en-us/research/wp-content/uploads/2017/07/Look-Closer-to-See-Better-Recurrent-Attention-Convolutional-Neural-Network-for-Fine-grained-Image-Recognition.pdf

https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Xiao_The_Application_of_2015_CVPR_paper.pdf

https://zhuanlan.zhihu.com/p/24738319?utm_campaign=rss&utm_content=title&utm_medium=rss&utm_source=rss